In [2]:
# Import necessary packages
import pandas as pd
import numpy as np
import time
import os
import pyodbc
import re
import datetime
import json
import warnings
import inspect
from datetime import timedelta
from slugify import slugify
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# Suppress warnings
warnings.simplefilter("ignore")

# Import NBA API endpoints
from nba_api.stats.endpoints.leaguegamelog import LeagueGameLog
from nba_api.stats.endpoints.playergamelogs import PlayerGameLogs
from nba_api.stats.endpoints.shotchartdetail import ShotChartDetail
from nba_api.stats.endpoints.boxscoreadvancedv3 import BoxScoreAdvancedV3
from nba_api.stats.endpoints.boxscoredefensivev2 import BoxScoreDefensiveV2
from nba_api.stats.endpoints.defensehub import DefenseHub
from nba_api.stats.endpoints.commonteamroster import CommonTeamRoster # params = season, team_id
from nba_api.stats.endpoints.boxscorematchupsv3 import BoxScoreMatchupsV3
from nba_api.stats.endpoints.matchupsrollup import MatchupsRollup
from nba_api.stats.endpoints.boxscorehustlev2 import BoxScoreHustleV2
from nba_api.stats.endpoints.boxscoremiscv3 import BoxScoreMiscV3
from nba_api.stats.endpoints.boxscoreplayertrackv3 import BoxScorePlayerTrackV3
from nba_api.stats.endpoints.boxscorescoringv3 import BoxScoreScoringV3
from nba_api.stats.endpoints.boxscoresummaryv2 import BoxScoreSummaryV2
from nba_api.stats.endpoints.teaminfocommon import TeamInfoCommon
from nba_api.stats.endpoints.infographicfanduelplayer import InfographicFanDuelPlayer
from nba_api.stats.endpoints.boxscorefourfactorsv3 import BoxScoreFourFactorsV3
from nba_api.stats.endpoints.boxscoreusagev3 import BoxScoreUsageV3

# Import the main nba_api.stats endpoints module
from nba_api.stats import endpoints


In [3]:
def pandas_to_sql_type(pandas_type):
    if pd.api.types.is_string_dtype(pandas_type):
        return 'NVARCHAR(MAX)'
    elif pd.api.types.is_integer_dtype(pandas_type):
        return 'INT'
    elif pd.api.types.is_float_dtype(pandas_type):
        return 'FLOAT'
    elif pd.api.types.is_bool_dtype(pandas_type):
        return 'BIT'
    elif pd.api.types.is_datetime64_dtype(pandas_type):
        return 'DATETIME'
    else:
        # Handle other data types as needed
        return 'NVARCHAR(MAX)'  # Default to NVARCHAR(MAX) for unsupported types

# Example usage:
# sql_type = pandas_to_sql_type(df[column].dtype)


def create_table_statement(df, table_name):
    columns = df.columns
    sql_columns = []
    for column in columns:
        sql_type = pandas_to_sql_type(df[column].dtype)
        sql_columns.append(f"[{column}] {sql_type}")

    columns_str = ",\n  ".join(sql_columns)
    sql_statement = f"CREATE TABLE {table_name} (\n  {columns_str}\n);"
    return sql_statement

In [4]:
today = datetime.date.today()
yearSeason = 2024
year='2023-24'

In [5]:
comparison_date = datetime.datetime.strptime(f"1/1/{yearSeason}", "%m/%d/%Y")

# Get the current date
today = datetime.datetime.now()

# Check if today's date is greater than the comparison date
is_greater = today > comparison_date
if is_greater:
    current_year = yearSeason - 1

In [6]:
server = 'localhost\SQLEXPRESS'
database = 'nba_game_data'

## League Game Log

In [9]:
# List of years to process
years = [
    #'2010-11', '2011-12','2012-13', '2013-14' ,'2014-15',
    '2015-16','2016-17', '2017-18', '2018-19', '2019-20',
    '2020-21','2021-22', '2022-23','2023-24'
]

final_game_log = pd.DataFrame()

# Loop through each year and fetch the game log data
for year in years:
    df = LeagueGameLog(season=year)
    shot_data = json.loads(df.get_json())
    relevant_data = shot_data['resultSets'][0]
    headers = relevant_data['headers']
    rows = relevant_data['rowSet']
    
    # Convert rows to DataFrame
    final_rows = pd.DataFrame(rows, columns=headers)
    final_rows = final_rows.fillna(0)
    final_rows['yearSeason'] = '20' + year.split('-')[1]
    
    # Append current year's data to the final DataFrame
    final_game_log = pd.concat([final_game_log, final_rows], ignore_index=True)

# Process the final DataFrame
final_game_log['GAME_DATE'] = pd.to_datetime(final_game_log['GAME_DATE'])
final_game_log['oppAbrv'] = final_game_log["MATCHUP"].apply(lambda x: x.split("@")[1].strip() if "@" in x else x.split("vs.")[1].strip())

In [10]:
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';')
cursor = cnxn.cursor()

cursor.execute("DELETE FROM leagueGameLog WHERE yearSeason = {} ".format(yearSeason))

# Prepare the INSERT SQL statement
insert_stmt = '''
INSERT INTO LeagueGameLog (SEASON_ID, TEAM_ID, TEAM_ABBREVIATION, TEAM_NAME, GAME_ID, GAME_DATE, MATCHUP, WL, MIN, FGM, FGA, FG_PCT, FG3M, FG3A, FG3_PCT, FTM, FTA, FT_PCT, OREB, DREB, REB, AST, STL, BLK, TOV, PF, PTS, PLUS_MINUS, VIDEO_AVAILABLE, yearSeason, oppAbrv)
VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
'''

# Iterate over DataFrame rows
for index, row in final_game_log.iterrows():
    try:
        # Execute the INSERT statement for each row
        cursor.execute(insert_stmt, tuple(row))
        cnxn.commit()
    except Exception as e:
        print(f"An error occurred at row {index}: {e}")
        # Optionally, break or continue based on the nature of the error

# Close the connection
cursor.close()
cnxn.close()


In [11]:
del df, final_game_log, final_rows, relevant_data, rows, shot_data

## PlayerGameLogs

In [13]:
final_player_game_log = pd.DataFrame()

# Assuming 'year' is defined elsewhere, e.g., year = '2023-24'
df = PlayerGameLogs(season_nullable=year, timeout=100)
shot_data = json.loads(df.get_json())
relevant_data = shot_data['resultSets'][0]
headers = relevant_data['headers']
rows = relevant_data['rowSet']

final_rows = pd.DataFrame(rows)
final_rows.columns = headers
final_rows = final_rows.fillna(0)
final_rows['yearSeason'] = '20' + year.split('-')[1]

# Use concat instead of append
final_player_game_log = pd.concat([final_player_game_log, final_rows], ignore_index=True)

final_player_game_log['oppAbrv'] = final_player_game_log["MATCHUP"].apply(lambda x: x.split("@")[1].strip() if "@" in x else x.split("vs.")[1].strip())


In [15]:
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';')
cursor = cnxn.cursor()

cursor.execute("DELETE FROM PlayerGameLogs WHERE yearSeason = {} ".format(yearSeason))

# Prepare the INSERT SQL statement
insert_stmt = '''
INSERT INTO PlayerGameLogs (SEASON_YEAR, PLAYER_ID, PLAYER_NAME, NICKNAME, TEAM_ID, TEAM_ABBREVIATION, TEAM_NAME, GAME_ID, GAME_DATE, MATCHUP, WL, MIN, FGM, FGA, FG_PCT, FG3M, FG3A, FG3_PCT, FTM, FTA, FT_PCT, OREB, DREB, REB, AST, TOV, STL, BLK, BLKA, PF, PFD, PTS, PLUS_MINUS, NBA_FANTASY_PTS, DD2, TD3, WNBA_FANTASY_PTS, GP_RANK, W_RANK, L_RANK, W_PCT_RANK, MIN_RANK, FGM_RANK, FGA_RANK, FG_PCT_RANK, FG3M_RANK, FG3A_RANK, FG3_PCT_RANK, FTM_RANK, FTA_RANK, FT_PCT_RANK, OREB_RANK, DREB_RANK, REB_RANK, AST_RANK, TOV_RANK, STL_RANK, BLK_RANK, BLKA_RANK, PF_RANK, PFD_RANK, PTS_RANK, PLUS_MINUS_RANK, NBA_FANTASY_PTS_RANK, DD2_RANK, TD3_RANK, WNBA_FANTASY_PTS_RANK, AVAILABLE_FLAG, yearSeason, oppAbrv)
VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
'''

# Iterate over DataFrame rows
for index, row in final_player_game_log.iterrows():
    try:
        # Execute the INSERT statement for each row
        cursor.execute(insert_stmt, tuple(row))
        cnxn.commit()
    except Exception as e:
        print(f"An error occurred at row {index}: {e}")
        # Optionally, break or continue based on the nature of the error

# Close the connection
cursor.close()
cnxn.close()

In [16]:
del df, final_player_game_log, final_rows, relevant_data, rows, shot_data

# Players Missed Games

In [17]:
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';')
cursor = cnxn.cursor()
sql = f"""
SELECT
       PlayerGameLogs.[PLAYER_ID]
      ,PlayerGameLogs.[PLAYER_NAME]
      ,PlayerGameLogs.[TEAM_ID]
      ,PlayerGameLogs.[GAME_ID]
      ,PlayerGameLogs.[GAME_DATE]
      ,PlayerGameLogs.yearSeason
      ,sum(case when trim(bsa.position) = '' then 0 else 1 end) over (partition by [PlayerGameLogs].PLAYER_ID, [PlayerGameLogs].SEASON_YEAR order by [PlayerGameLogs].GAME_DATE) as numberOfGamesStarted

      
  FROM [nba_game_data].[dbo].[PlayerGameLogs]
  
  LEFT OUTER JOIN [nba_game_data].[dbo].[BoxScoreAdvancedV3] bsa
  on cast([PlayerGameLogs].PLAYER_ID as int) = cast(bsa.personId as int)
  and cast([PlayerGameLogs].GAME_ID as int) = cast(bsa.GAME_ID as int)
  
  WHERE PlayerGameLogs.[yearSeason] = '{yearSeason}'
  order by PlayerGameLogs.PLAYER_ID, PlayerGameLogs.GAME_DATE desc
"""
df = pd.read_sql(sql,cnxn)
# df

In [18]:
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';')
cursor = cnxn.cursor()
sql =f'''SELECT [SEASON_ID]
      ,[TEAM_ID]
      ,[TEAM_ABBREVIATION]
      ,[TEAM_NAME]
      ,[GAME_ID]
      ,[GAME_DATE]
      ,yearSeason
  FROM [nba_game_data].[dbo].[LeagueGameLog]
  WHERE [yearSeason] = '{yearSeason}'
  '''
team_games = pd.read_sql(sql,cnxn)
team_games['GAME_DATE'] = pd.to_datetime(team_games['GAME_DATE'])

In [19]:
# Convert GAME_DATE columns to datetime
df['GAME_DATE'] = pd.to_datetime(df['GAME_DATE'])
team_games['GAME_DATE'] = pd.to_datetime(team_games['GAME_DATE'])

# Ensure df is sorted by GAME_DATE
df = df.sort_values(by='GAME_DATE')

# Create a new DataFrame to hold the expanded player game records
expanded_player_records = pd.DataFrame()

# For each PLAYER_ID in df
for player_id in df['PLAYER_ID'].unique():
    player_df = df[df['PLAYER_ID'] == player_id]
    
    # Generate a date range from the player's first game to their last game
    min_date = player_df['GAME_DATE'].min()
    max_date = player_df['GAME_DATE'].max()
    all_dates = pd.date_range(start=min_date, end=max_date)
    
    # Create a DataFrame from this date range
    date_df = pd.DataFrame(all_dates, columns=['GAME_DATE'])
    
    # Add back the PLAYER_ID
    date_df['PLAYER_ID'] = player_id
    
    # Merge to include GAME_ID where it exists, and forward fill TEAM_ID
    merged_df = pd.merge_asof(date_df.sort_values('GAME_DATE'), player_df[['GAME_DATE', 'TEAM_ID', 'GAME_ID']].sort_values('GAME_DATE'), on='GAME_DATE', direction='forward')
    
    # Add an indicator for whether the GAME_ID was filled in or existed in the original df
    # 'existing' if the row was in the original df, 'filled' if it was added during the process
    merged_df['record_type'] = merged_df['GAME_ID'].notna().map({True: 'existing', False: 'filled'})
    
    # Forward fill the TEAM_ID and GAME_ID for continuity, but only forward fill GAME_ID for 'existing' records
    merged_df['TEAM_ID'] = merged_df['TEAM_ID'].ffill()
    merged_df.loc[merged_df['record_type'] == 'existing', 'GAME_ID'] = merged_df.loc[merged_df['record_type'] == 'existing', 'GAME_ID'].ffill()

    # Append to the expanded player records DataFrame
    expanded_player_records = pd.concat([expanded_player_records, merged_df], ignore_index=True)

# Correct 'record_type' for days without games to reflect accurate status
expanded_player_records.loc[expanded_player_records['GAME_ID'].isna(), 'record_type'] = 'filled'

final_games_missed = pd.merge(expanded_player_records,df, on = ['GAME_DATE','PLAYER_ID','TEAM_ID'], how ='left')
final_games_missed = pd.merge(final_games_missed,team_games, on = ['GAME_DATE','TEAM_ID'], how ='inner')
final_games_missed = final_games_missed.fillna(-999)

final_games_missed['DID_PLAYER_MISS_GAME'] = (final_games_missed['GAME_ID_y'] == -999).astype(int)

final_games_missed = final_games_missed[['PLAYER_ID','GAME_DATE','GAME_ID','DID_PLAYER_MISS_GAME']]


In [21]:
# Assuming 'server' and 'database' variables are defined elsewhere, along with 'final_games_missed' DataFrame
cnxn = pyodbc.connect(f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};')
cursor = cnxn.cursor()

cursor.execute("DELETE FROM GamesPlayerMissedV1")
cnxn.commit()

# Check if there are records to insert
if len(final_games_missed) > 0:
    # SQL INSERT statement for the 'GamesPlayerMissedV1' table
    insert_stmt = '''
    INSERT INTO GamesPlayerMissedV1 (PLAYER_ID, GAME_DATE, GAME_ID, DID_PLAYER_MISS_GAME)
    VALUES (?, ?, ?, ?)
    '''

    # Insert rows into the database table
    for index, row in final_games_missed.iterrows():
        try:
            # Convert the relevant parts of the row to a tuple and execute the INSERT statement
            # Ensure the order of row's elements matches the order in the insert_stmt
            data_tuple = (
                int(row['PLAYER_ID']),
                row['GAME_DATE'],  # Assuming GAME_DATE is already a datetime object; no conversion needed
                int(row['GAME_ID']),
                int(row['DID_PLAYER_MISS_GAME'])
            )
            cursor.execute(insert_stmt, data_tuple)
            cnxn.commit()
        except Exception as e:
            print(f"An error occurred at row {index}: {e}")
            # Depending on your error handling preference, you can choose to break or continue

    cursor.close()
    cnxn.close()
else:
    print("No data to insert.")


## Some testing stuff

In [22]:
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';')
cursor = cnxn.cursor()
sql = """
/****** Script for SelectTopNRows command from SSMS  ******/
SELECT
	   pgl.[PLAYER_ID]
      ,pgl.[PLAYER_NAME]
	  ,COALESCE(CASE WHEN ctr.[POSITION] = 'G-F' THEN 'G' ELSE
        CASE WHEN ctr.[POSITION] = 'F-C' THEN 'F' ELSE
        CASE WHEN ctr.[POSITION] = 'F-G' THEN 'F' ELSE
        CASE WHEN ctr.[POSITION] = 'C-F' THEN 'C' ELSE
        ctr.[POSITION] END END END END ,ctr2.POSITION,'NF') as POSITION

      ,avg(pgl.PTS + pgl.REB + pgl.AST) as pts_reb_astPerGame
      
      ,avg(cast(pgl.[PTS] as Float))  PtsPerGamePerGame
      ,avg(cast(pgl.[FG_PCT] as Float)) FGPctPerGame
      ,avg(cast(pgl.[FG3_PCT] as Float)) FG3PctPerGame
	  ,avg(cast(bsa.[effectiveFieldGoalPercentage] as Float)) EfgPctPerGame
      ,avg(cast(bsa.[trueShootingPercentage] as Float)) TSPctPerGame
	  ,avg(cast(bsh.[contestedShots] as Float)) ContShotPerGame
      ,avg(cast(bsh.[contestedShots2pt] as Float)) ContShot2PerGame
      ,avg(cast(bsh.[contestedShots3pt] as Float)) ContShot3PerGame
	   
      ,avg(cast(pgl.[OREB] as Float)) orebPerGame
      ,avg(cast(pgl.[DREB] as Float)) drebPerGame
      ,avg(cast(pgl.[REB] as Float)) rebPerGame
	  ,avg(cast(bsa.[offensiveReboundPercentage] as Float)) orbPctPerGame
      ,avg(cast(bsa.[defensiveReboundPercentage] as Float)) drbPctPerGame
      ,avg(cast(bsa.[reboundPercentage] as Float)) rPctPerGame
	  ,avg(cast(bsh.[screenAssistPoints] as Float)) screenAstPtsPerGame
	  ,avg(cast(bsh.[offensiveBoxOuts] as Float)) as oboPerGame
      ,avg(cast(bsh.[defensiveBoxOuts] as Float)) dboPerGame
      ,avg(cast(bsh.[boxOutPlayerTeamRebounds] as Float)) boptrebPerGame
      ,avg(cast(bsh.[boxOutPlayerRebounds] as Float)) boprebPerGame
      ,avg(cast(bsh.[boxOuts] as Float)) boPerGame
	   	  
      ,avg(cast(pgl.[AST] as Float)) astPerGame
	  ,avg(cast(bsa.[assistPercentage] as Float)) astPctPerGame
      ,avg(cast(bsa.[assistToTurnover] as Float)) asttoTovPerGame
      ,avg(cast(bsa.[assistRatio] as Float)) astRioPerGame
	  ,avg(cast(bsh.[screenAssists] as Float)) screenAstPerGame
      
	   	   
      ,avg(cast(pgl.[TOV] as Float)) TovPerGame
	  ,avg(cast(bsa.[turnoverRatio] as Float)) tovRtiPerGame
	   	   
      ,avg(cast(pgl.[STL] as Float)) stlPerGame
	  ,avg(cast(bsh.[deflections] as Float)) deflPerGame
	  ,avg(cast(bsh.[looseBallsRecoveredOffensive] as Float)) olooseRecPerGame
      ,avg(cast(bsh.[looseBallsRecoveredDefensive] as Float)) dlooseRecPerGame
	   	   
      ,avg(cast(pgl.[BLK] as Float)) blkPerGame
      ,avg(cast(pgl.[BLKA] as Float)) blkaPerGame

,avg(cast(pgl.[MIN] as Float)) minPerGame
      ,avg(cast(pgl.[PLUS_MINUS] as Float)) plusminusPerGame
	  ,avg(cast(bsa.[PIE] as Float)) piePerGame
	  ,avg(cast(bsa.[netRating] as Float)) netratingPerGame
	  ,avg(cast(bsa.[defensiveRating] as Float)) defratingPerGame
	  ,avg(cast(bsa.[usagePercentage] as Float)) usagePctPerGame
	  ,avg(cast([looseBallsRecoveredTotal] as Float)) loosBallRecPerGame


  FROM [nba_game_data].[dbo].[PlayerGameLogs] pgl 


  LEFT OUTER JOIN [nba_game_data].[dbo].[BoxScoreAdvancedV3] bsa 
  on 
  pgl.GAME_ID = bsa.GAME_ID
  and pgl.PLAYER_ID = bsa.personId

    LEFT OUTER JOIN  [nba_game_data].[dbo].[BoxScoreHustleV2] bsh 
  on 
  pgl.GAME_ID = bsh.gameId
  and pgl.PLAYER_ID = bsh.personId
  LEFT OUTER JOIN [nba_game_data].[dbo].[CommonTeamRoster] ctr
  ON
  cast(pgl.PLAYER_ID as int) = cast(ctr.PLAYER_ID as int)
  AND cast(pgl.TEAM_ID as int) = cast(ctr.TeamID as int)
  AND cast(pgl.yearSeason as int) = cast(ctr.SEASON as int)
  
  LEFT OUTER JOIN [nba_game_data].[dbo].[BackupPlayerPosition] ctr2
  ON 
  cast(pgl.PLAYER_ID as int) = cast(ctr2.PLAYER_ID as int)
  
  where pgl.yearSeason = 2024

  group by pgl.[PLAYER_ID]
      ,pgl.[PLAYER_NAME]
	  ,COALESCE(CASE WHEN ctr.[POSITION] = 'G-F' THEN 'G' ELSE
        CASE WHEN ctr.[POSITION] = 'F-C' THEN 'F' ELSE
        CASE WHEN ctr.[POSITION] = 'F-G' THEN 'F' ELSE
        CASE WHEN ctr.[POSITION] = 'C-F' THEN 'C' ELSE
        ctr.[POSITION] END END END END ,ctr2.POSITION,'NF')
        

  """
df1 = pd.read_sql(sql,cnxn)

import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# Function to perform KMeans clustering and add cluster labels to the dataframe
def perform_kmeans(df, features, n_clusters=40, weight_first_element=8):
    # Extracting the features
    df_features = df[features]
    
    # Standardizing the features except the first one
    scaler = StandardScaler()
    df_scaled = scaler.fit_transform(df_features.iloc[:, 1:])  # Skip the first feature for now
    
    # Scaling the first feature separately with a higher weight
    first_feature_scaled = df_features.iloc[:, 0].values.reshape(-1, 1) * weight_first_element
    first_feature_scaled = scaler.fit_transform(first_feature_scaled)  # Standardize the weighted first feature
    
    # Combining the weighted first feature with the other scaled features
    df_scaled = np.hstack((first_feature_scaled, df_scaled))
    
    # KMeans clustering
    kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(df_scaled)
    return kmeans.labels_

# Dictionary to hold features for each stat category
features_dict = {
    'PTS': ['PtsPerGamePerGame','FGPctPerGame', 'EfgPctPerGame', 'TSPctPerGame', 'ContShot2PerGame', 'ContShot3PerGame', 'minPerGame', 'usagePctPerGame'],
    'REB': ['rebPerGame','orbPctPerGame', 'drbPctPerGame', 'rPctPerGame', 'minPerGame', 'piePerGame', 'loosBallRecPerGame'],
    'AST': ['astPerGame','astPctPerGame', 'asttoTovPerGame', 'minPerGame', 'defratingPerGame', 'loosBallRecPerGame'],
    'STL': ['stlPerGame','dlooseRecPerGame', 'olooseRecPerGame', 'deflPerGame', 'piePerGame', 'netratingPerGame', 'loosBallRecPerGame'],
    'BLK': ['blkPerGame','blkaPerGame', 'boPerGame', 'deflPerGame', 'piePerGame', 'netratingPerGame', 'loosBallRecPerGame'],
    'TOV': ['TovPerGame','dlooseRecPerGame', 'olooseRecPerGame', 'deflPerGame', 'piePerGame', 'netratingPerGame', 'loosBallRecPerGame'],
    'FG3M': ['FG3PctPerGame','EfgPctPerGame', 'TSPctPerGame', 'ContShot2PerGame', 'ContShot3PerGame', 'minPerGame', 'usagePctPerGame'],
    'PTS_REB_AST': ['pts_reb_astPerGame','PtsPerGamePerGame','rebPerGame', 'astPerGame','TSPctPerGame','minPerGame', 'usagePctPerGame','piePerGame','loosBallRecPerGame']
}

# Assuming df is your dataframe from the SQL query
# Add a column for each cluster label for each stat
for stat, features in features_dict.items():
    df1[f'{stat}Cluster'] = None  # Initialize column for cluster labels
#     stat_pos = 1
    for position in df1['POSITION'].unique():
        # Filter dataframe for the current position
        position_df = df1[df1['POSITION'] == position]
        position_df = position_df.fillna(0)
        if position_df.empty:
            continue  # Skip if no players for the position
        if position == 'NF':
            cluster_labels = perform_kmeans(position_df, features,7)
        else:
            # Perform KMeans clustering for the current position and stat
            cluster_labels = perform_kmeans(position_df, features)
        
        max_cluster = df1[f'{stat}Cluster'].max()
        if np.isnan(max_cluster):
            max_cluster =0
        
        cluster_labels = cluster_labels + max_cluster
        
        
        # Assign cluster labels back to the main dataframe
        df1.loc[df1['POSITION'] == position, f'{stat}Cluster'] = cluster_labels

# Select relevant columns for the final output
output_columns = ['PLAYER_ID', 'PLAYER_NAME', 'POSITION'] + [f'{stat}Cluster' for stat in features_dict]
final_output = df1[output_columns]

# final_output

In [24]:
cnxn = pyodbc.connect(f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};')
cursor = cnxn.cursor()

# Clear the table only if you're sure you want to remove existing data
cursor.execute("DELETE FROM player_clustering")
cnxn.commit()

# Check if there are records to insert in the 'final_output' DataFrame
if not final_output.empty:
    insert_stmt = '''
    INSERT INTO player_clustering (PLAYER_ID, PLAYER_NAME, POSITION, PTSCluster, REBCluster, ASTCluster, STLCluster, BLKCluster, TOVCluster, FG3MCluster, PTS_REB_ASTCluster)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    '''

    # Insert rows into the database table
    for index, row in final_output.iterrows():
        try:
            # Convert the relevant parts of the row to a tuple and execute the INSERT statement
            # Ensure the order of row's elements matches the order in the insert_stmt
            data_tuple = (
                int(row['PLAYER_ID']),  # Assuming PLAYER_ID is an integer
                row['PLAYER_NAME'],  # Assuming PLAYER_NAME is a string
                row['POSITION'],  # Assuming POSITION is a string
                str(row['PTSCluster']),  # Convert cluster numbers to string if necessary
                str(row['REBCluster']),  # Convert cluster numbers to string if necessary
                str(row['ASTCluster']),  # Convert cluster numbers to string if necessary
                str(row['STLCluster']),  # Convert cluster numbers to string if necessary
                str(row['BLKCluster']),  # Convert cluster numbers to string if necessary
                str(row['TOVCluster']),  # Convert cluster numbers to string if necessary
                str(row['FG3MCluster']),  # Convert cluster numbers to string if necessary
                str(row['PTS_REB_ASTCluster'])  # Convert cluster numbers to string if necessary
            )
            cursor.execute(insert_stmt, data_tuple)
            cnxn.commit()
        except Exception as e:
            print(f"An error occurred at row {index}: {e}")
            # Depending on your error handling preference, you might choose to break or continue

    cursor.close()
    cnxn.close()
else:
    print("No data to insert.")


In [25]:
test1 = pd.merge(expanded_player_records,df, on = ['GAME_DATE','PLAYER_ID','TEAM_ID'], how ='left')
test = pd.merge(test1,team_games, on = ['GAME_DATE','TEAM_ID'], how ='inner')
test = test.fillna(-999)

test['DID_PLAYER_MISS_GAME'] = (test['GAME_ID_y'] == -999).astype(int)

test = test[['PLAYER_ID','GAME_DATE','GAME_ID','DID_PLAYER_MISS_GAME']]
# test.loc[test['PLAYER_ID']==201144].iloc[-60:]

In [26]:
# Assume df is your initial DataFrame and GAME_DATE has been converted to datetime
df['GAME_DATE'] = pd.to_datetime(df['GAME_DATE'])

# First, determine the max date for each team and season combination
team_max_dates = df.groupby(['TEAM_ID', 'yearSeason'])['GAME_DATE'].max().reset_index(name='TEAM_MAX_DATE')

# Merge this max date back into the original DataFrame
df = pd.merge(df, team_max_dates, on=['TEAM_ID', 'yearSeason'])

def fill_in_dates(group):
    # Create a date range for the group, now using the team's max date
    date_range = pd.date_range(start=group['GAME_DATE'].min(), end=group['TEAM_MAX_DATE'].max())
    
    # Create a DataFrame from the date range
    date_df = pd.DataFrame(date_range, columns=['FULL_DATE'])
    
    # Merge with the original group, indicating where the data came from
    merged_df = pd.merge(date_df, group, left_on='FULL_DATE', right_on='GAME_DATE', how='left', indicator=True)
    
    # The '_merge' column will say 'left_only' for dates that were not in the original data
    merged_df['WAS_PLAYER_OUT'] = (merged_df['_merge'] == 'left_only').astype(int)
    
    # Clean up the DataFrame, dropping unnecessary columns and filling in missing values
    merged_df.drop(columns=['_merge', 'TEAM_MAX_DATE'], inplace=True)  # Drop the TEAM_MAX_DATE column after use
    merged_df['PLAYER_ID'].fillna(method='ffill', inplace=True)
    merged_df['PLAYER_NAME'].fillna(method='ffill', inplace=True)
    merged_df['TEAM_ID'].fillna(method='ffill', inplace=True)
    
    return merged_df

# Apply the function to each group of data by player and yearSeason
filled_dates_df = df.groupby(['PLAYER_ID', 'yearSeason']).apply(fill_in_dates).reset_index(drop=True)

# Ensure TEAM_ID is the correct data type
filled_dates_df['TEAM_ID'] = filled_dates_df['TEAM_ID'].astype('int64')


In [27]:
final_player_play = pd.merge(team_games, filled_dates_df[['FULL_DATE','PLAYER_ID','TEAM_ID','WAS_PLAYER_OUT','numberOfGamesStarted']], left_on=['TEAM_ID','GAME_DATE'], right_on=['TEAM_ID','FULL_DATE'], how='inner').sort_values(['PLAYER_ID','GAME_DATE'])

In [28]:
final_player_play['numberOfGamesStarted'] = final_player_play.groupby(['PLAYER_ID', 'yearSeason'])['numberOfGamesStarted'].ffill()
# final_player_play

In [29]:
final_player_play['numberOfGamesStarted'] = final_player_play['numberOfGamesStarted'].fillna(0).astype(int)

In [30]:
final_player_play['WAS_STARTER_OUT'] = ((final_player_play['WAS_PLAYER_OUT'] > 0) & (final_player_play['numberOfGamesStarted'] > 0)).astype(int)
# final_player_play

In [31]:
# Grouping by TEAM_ID and GAME_ID to calculate the number of players out and starters out
grouped_data = final_player_play.groupby(['TEAM_ID', 'GAME_ID','yearSeason']).agg(
    total_players_out=('WAS_PLAYER_OUT', 'sum'),
    total_starters_out=('WAS_STARTER_OUT', 'sum')
).reset_index()

grouped_data['total_non_starters_out'] = grouped_data['total_players_out'] - grouped_data['total_starters_out']

In [32]:
if len(grouped_data) > 0:
    # Presuming 'server' and 'database' variables are defined elsewhere
    cnxn = pyodbc.connect(f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};')
    cursor = cnxn.cursor()
    
    cursor.execute("DELETE FROM dailyInjuryReportV1 WHERE yearSeason = {} ".format(yearSeason))
    cnxn.commit()
    
    # Assuming you don't need to delete existing records for this operation, we skip that part
    # If you do need to delete or update existing records, include those operations here

    # SQL INSERT statement for the 'dailyInjuryReportV1' table
    insert_stmt = '''
    INSERT INTO dailyInjuryReportV1 (TEAM_ID, GAME_ID, yearSeason,total_players_out, total_starters_out, total_non_starters_out)
    VALUES (?, ?, ?, ?, ?, ?)
    '''

    # Insert rows into the database table
    for index, row in grouped_data.iterrows():
        try:
            # Convert the relevant parts of the row to a tuple and execute the INSERT statement
            # Make sure the order of row's elements matches the order in the insert_stmt
            data_tuple = (
                int(row['TEAM_ID']), 
                int(row['GAME_ID']),
                int(row['yearSeason']), 
                int(row['total_players_out']), 
                int(row['total_starters_out']), 
                int(row['total_non_starters_out'])
            )        
            cursor.execute(insert_stmt, data_tuple)
            cnxn.commit()
        except Exception as e:
            print(f"An error occurred at row {index}: {e}")
#             # You can choose to either break or continue based on your preference

    cursor.close()
    cnxn.close()



In [33]:
# Convert GAME_DATE to datetime to ensure proper sorting
df['GAME_DATE'] = pd.to_datetime(df['GAME_DATE'])

# Sort the DataFrame by PLAYER_ID and GAME_DATE
df = df.sort_values(by=['PLAYER_ID', 'GAME_DATE'])

# Identify trades
df['traded'] = ((df['TEAM_ID'] != df['TEAM_ID'].shift(1)) & (df['PLAYER_ID'] == df['PLAYER_ID'].shift(1))).astype(int)


In [34]:
df = df.reset_index(drop=True)

In [35]:
def mark_after_trade(group):
    # Find indices where trades happened
    trade_indices = group.index[group['traded'] == 1].tolist()
    # For each index where a trade happened, mark all following rows in the group
    for idx in trade_indices:
        # Mark rows after a trade with 1
        group.loc[idx + 1:, 'after_trade'] = 1
    return group

# Initialize the after_trade column with 0
df['after_trade'] = df['traded']

# Apply the function to each player-season group
df = df.groupby(['PLAYER_ID', 'yearSeason']).apply(mark_after_trade)

# df

In [39]:
df = df.reset_index(drop=True)

In [40]:
# Ensure the GAME_DATE is in datetime format (if not already)
df['GAME_DATE'] = pd.to_datetime(df['GAME_DATE'])

# Sort the DataFrame by PLAYER_ID and GAME_DATE just to be sure
df = df.sort_values(by=['PLAYER_ID', 'GAME_DATE'])

df = df.reset_index(drop=True)

# Initialize the column with zeros
df['traded_in_last_10_games'] = 0

# Define a function to apply rolling operation for each player
def calculate_traded_in_last_10_games_corrected(group):
    # Calculate rolling sum of trades in the last 10 games NOT including the current game
    # We use window=10, closed='left' to not include the current row in the sum
    group['temp_rolling_trades'] = group['traded'].rolling(window=10, min_periods=1, closed='left').sum()
    # Fill NaN values with 0 before adding the current game's traded status
    group['temp_rolling_trades'].fillna(0, inplace=True)
    # Now, we add the current game's traded status to the rolling sum to include it in our criteria
    group['traded_in_last_10_games'] = group['temp_rolling_trades'] + group['traded']
    # Cleanup: drop the temporary column
    group.drop(columns=['temp_rolling_trades'], inplace=True)
    return group

# Apply the function to each player group
df = df.groupby('PLAYER_ID').apply(calculate_traded_in_last_10_games_corrected)

# Ensure the 'traded_in_last_10_games' column is integer type
df['traded_in_last_10_games'] = df['traded_in_last_10_games'].astype(int)


In [41]:
df = df[['PLAYER_ID','TEAM_ID','GAME_ID','GAME_DATE','traded','after_trade','traded_in_last_10_games','yearSeason']]

In [42]:
if len(df) > 0:
    cnxn = pyodbc.connect(f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};')
    cursor = cnxn.cursor()
    
    # Delete existing records for the given yearSeason in the TradeReportV1 table
    cursor.execute("DELETE FROM TradeReportV1 WHERE yearSeason = '{}' ".format(yearSeason))
    cnxn.commit()
    
    # SQL INSERT statement for the 'TradeReportV1' table
    insert_stmt = '''
    INSERT INTO TradeReportV1 (PLAYER_ID, TEAM_ID, GAME_ID, GAME_DATE, traded, after_trade, traded_in_last_10_games, yearSeason)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?)
    '''

    # Insert rows into the TradeReportV1 table
    for index, row in df.iterrows():
        try:
            # Ensure the date format matches your database requirements. Adjust as necessary.
            game_date = pd.to_datetime(row['GAME_DATE']).strftime('%Y-%m-%d %H:%M:%S')
            
            # Convert the relevant parts of the row to a tuple and execute the INSERT statement
            data_tuple = (
                int(row['PLAYER_ID']),
                int(row['TEAM_ID']),
                int(row['GAME_ID']),
                game_date, # Assuming 'GAME_DATE' is already in the correct DATETIME format
                int(row['traded']),
                int(row['after_trade']),
                int(row['traded_in_last_10_games']),
                str(row['yearSeason']) # Assuming 'yearSeason' is a string, adjust if it's numeric
            )
            cursor.execute(insert_stmt, data_tuple)
            cnxn.commit()
        except Exception as e:
            print(f"An error occurred at row {index}: {e}")
            # Decide whether to break or continue based on your error handling preference

    cursor.close()
    cnxn.close()


## ShotChartDetail

In [ ]:
# cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';')
# cursor = cnxn.cursor()
# sql = 'SELECT distinct player_id, team_id, season_year FROM [nba_game_data].[dbo].[PlayerGameLogs] where yearSeason = {}'.format(yearSeason)
# all_players = pd.read_sql(sql,cnxn)  
# cnxn.close()

In [ ]:
# final_shooting = pd.DataFrame()

# for index, row in all_players.iterrows():
#     try:
#         df = ShotChartDetail(
#                 team_id = row['team_id'],
#                 player_id = row['player_id'],
#                 context_measure_simple = 'FGM',
#                     season_nullable = row['season_year'],
#                 season_type_all_star = 'Regular Season')
#         shot_data = json.loads(df.get_json())
#         relevant_data = shot_data['resultSets'][0]
#         headers = relevant_data['headers']
#         rows = relevant_data['rowSet']
#         final_rows = pd.DataFrame(rows)
#         final_rows.columns = headers
#         final_shooting = final_shooting.append(final_rows).reset_index(drop=True)
#     except:
#         pass

In [ ]:
# cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';')
# cursor = cnxn.cursor()

# cursor.execute("DELETE FROM ShotChartDetail where [GAME_DATE] > {}0701".format(yearSeason-1))

# # Prepare the INSERT SQL statement
# insert_stmt = '''
# INSERT INTO ShotChartDetail (GRID_TYPE, GAME_ID, GAME_EVENT_ID, PLAYER_ID, PLAYER_NAME, TEAM_ID, TEAM_NAME, PERIOD, MINUTES_REMAINING, SECONDS_REMAINING, EVENT_TYPE, ACTION_TYPE, SHOT_TYPE, SHOT_ZONE_BASIC, SHOT_ZONE_AREA, SHOT_ZONE_RANGE, SHOT_DISTANCE, LOC_X, LOC_Y, SHOT_ATTEMPTED_FLAG, SHOT_MADE_FLAG, GAME_DATE, HTM, VTM)
# VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
# '''

# # Iterate over DataFrame rows
# for index, row in final_shooting.iterrows():
#     try:
#         # Execute the INSERT statement for each row
#         cursor.execute(insert_stmt, tuple(row))
#         cnxn.commit()
#     except Exception as e:
#         print(f"An error occurred at row {index}: {e}")
#         # Optionally, break or continue based on the nature of the error

# # Close the connection
# cursor.close()
# cnxn.close()


In [ ]:
# del df, all_players, final_shooting, relevant_data, rows, shot_data, headers, final_rows

## InfluentialPlayersV1

In [44]:
def get_most_import_players(line_type):
    server = 'localhost\SQLEXPRESS'
    database = 'nba_game_data'

    sql = f'''
    with minutesPerGame as (
    SELECT [SEASON_YEAR]
          ,[PLAYER_ID]
          ,avg(MIN) avg_min
      FROM [nba_game_data].[dbo].[PlayerGameLogs]
      where yearSeason =2024
      group by [SEASON_YEAR]
          ,[PLAYER_ID]
    )

    SELECT pgl.[SEASON_YEAR]
          ,pgl.[PLAYER_ID]
          ,pgl.[PLAYER_NAME]
          ,pgl.[NICKNAME]
          ,pgl.[TEAM_ID]
          ,pgl.[TEAM_ABBREVIATION]
          ,pgl.[TEAM_NAME]
          ,pgl.[GAME_ID]
          ,pgl.[GAME_DATE]

          ,{line_type} as PTS
      FROM [nba_game_data].[dbo].[PlayerGameLogs] pgl
       left outer join minutesPerGame mpg
      on 
      pgl.[PLAYER_ID] = mpg.[PLAYER_ID]
      and pgl.[SEASON_YEAR] = mpg.[SEASON_YEAR]

      where yearSeason =2024
      and  mpg.avg_min > 20
    '''
    cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';')
    cursor = cnxn.cursor()
    df = pd.read_sql(sql,cnxn)
    
    sql = f'''
    SELECT *
      FROM PlayersCurrentTeam
    '''
    cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';')
    cursor = cnxn.cursor()
    players_current_team = pd.read_sql(sql,cnxn)

    average_pts = df.groupby('PLAYER_NAME')['PTS'].mean().reset_index()

    # Rename the columns for clarity
    average_pts.rename(columns={'PTS': 'AVG_PTS'}, inplace=True)
    
    df_with_avg = pd.merge(df, average_pts, on='PLAYER_NAME')
    
    teammates = df_with_avg.groupby(['TEAM_ID', 'GAME_ID'])['PLAYER_ID'].apply(list).reset_index()
    teammates = teammates.rename(columns={'PLAYER_ID':'Players_Who_Played_In_That_Game'})
    teammate_joins = df_with_avg.merge(teammates, on=['TEAM_ID','GAME_ID'])
    
#     teammate_joins['All_Teammates'] = teammate_joins['PLAYER_ID'].apply(lambda x: list(teammates_dict[x]) if x in teammates_dict else [])
    
    teammate_joins['Players_Who_Played_In_That_Game'] = teammate_joins['Players_Who_Played_In_That_Game'].apply(
        lambda x: literal_eval(x) if isinstance(x, str) else x
    )

    # Proceed with creating the set of all teammates
    teammates_dict = {}
    
    teammate_joins['All_Teammates'] = teammate_joins['PLAYER_ID'].apply(lambda x: list(teammates_dict[x]) if x in teammates_dict else [])
    
    for index, row in teammate_joins.iterrows():
        player_id = row['PLAYER_ID']
        teammates = set(row['Players_Who_Played_In_That_Game']) - {player_id}  # Remove the player's own ID
        if player_id not in teammates_dict:
            teammates_dict[player_id] = teammates
        else:
            teammates_dict[player_id].update(teammates)

    # Map the teammates set back to the DataFrame, creating a new column 'All_Teammates'
    teammate_joins['All_Teammates'] = teammate_joins['PLAYER_ID'].apply(lambda x: list(teammates_dict[x]))

    # Ensure no duplication within 'All_Teammates'
    teammate_joins['All_Teammates'] = teammate_joins['All_Teammates'].apply(lambda x: list(set(x)))
    
    player_to_team_map = players_current_team.set_index('PLAYER_ID')['TEAM_ABBREVIATION'].to_dict()

    # Placeholder DataFrame for importance scores and games played
    importance_scores_df = pd.DataFrame(columns=['PLAYER_ID', 'TEAMMATE_ID','RAW_IMPORTANCE_SCORE', 'IMPORTANCE_SCORE', 'GAMES_PLAYED','TOTAL_GAMES_PLAYED'])

    for player_id in teammate_joins['PLAYER_ID'].unique():
        player_games = teammate_joins[teammate_joins['PLAYER_ID'] == player_id]

        number_of_games = len(player_games)

        current_team_id = player_to_team_map.get(player_id)
        all_teammates = set()

        for index, row in player_games.iterrows():
            all_teammates.update(row['All_Teammates'])

        for teammate_id in all_teammates:
            teammate_current_team_id = player_to_team_map.get(teammate_id)
            if teammate_current_team_id != current_team_id:
                continue

            pts_with_teammate = []
            pts_without_teammate = []
            games_with_teammate = 0

            for index, game in player_games.iterrows():
                if teammate_id in game['Players_Who_Played_In_That_Game']:
                    pts_with_teammate.append(game['PTS'] - game['AVG_PTS'])
                    games_with_teammate += 1
                else:
                    pts_without_teammate.append(game['PTS'] - game['AVG_PTS'])

            avg_diff_with = sum(pts_with_teammate) / len(pts_with_teammate) if pts_with_teammate else 0
            avg_diff_without = sum(pts_without_teammate) / len(pts_without_teammate) if pts_without_teammate else 0

            importance_score = avg_diff_with - avg_diff_without

            temp = pd.DataFrame([{
                'PLAYER_ID': player_id,
                'TEAMMATE_ID': teammate_id,
                'RAW_IMPORTANCE_SCORE':importance_score,
                'IMPORTANCE_SCORE': abs(importance_score),
                'GAMES_PLAYED': games_with_teammate,
                'TOTAL_GAMES_PLAYED': number_of_games
            }])

            importance_scores_df = pd.concat([importance_scores_df, temp], ignore_index=True)



    importance_scores_df['PCT_OF_GAMES_PLAYED_TOGETHER'] = importance_scores_df['GAMES_PLAYED']/importance_scores_df['TOTAL_GAMES_PLAYED']
    importance_scores_df = importance_scores_df.loc[importance_scores_df['PCT_OF_GAMES_PLAYED_TOGETHER']>=.25].reset_index(drop=True)
    
    importance_scores_df = importance_scores_df.sort_values(by=['PLAYER_ID', 'IMPORTANCE_SCORE'], ascending=[True, False])
    importance_scores_df = importance_scores_df.drop_duplicates(subset=['PLAYER_ID'], keep='first')
    
    importance_scores_df = importance_scores_df[['PLAYER_ID','TEAMMATE_ID','RAW_IMPORTANCE_SCORE']]
    return importance_scores_df.rename(columns={'TEAMMATE_ID' : f'{line_type}_TEAMMATE_ID','RAW_IMPORTANCE_SCORE':f'{line_type}_IMPORTANCE_SCORE'})

In [45]:
tracker = 0 
lines = ['PTS','REB','AST','STL','BLK','TOV','FG3M','DD2']
for i in lines:
    if tracker == 0:
        output =get_most_import_players(i)
        tracker +=1
    else:
        output = pd.merge(output,get_most_import_players(i),on='PLAYER_ID')

,PLAYER_ID,PTS_TEAMMATE_ID,PTS_IMPORTANCE_SCORE,REB_TEAMMATE_ID,REB_IMPORTANCE_SCORE,AST_TEAMMATE_ID,AST_IMPORTANCE_SCORE,STL_TEAMMATE_ID,STL_IMPORTANCE_SCORE,BLK_TEAMMATE_ID,BLK_IMPORTANCE_SCORE,TOV_TEAMMATE_ID,TOV_IMPORTANCE_SCORE,FG3M_TEAMMATE_ID,FG3M_IMPORTANCE_SCORE,DD2_TEAMMATE_ID,DD2_IMPORTANCE_SCORE
0,2544,1629060,4.294486,203076,-1.403030,1627752,2.697761,1629060,-0.307018,1629060,-0.401003,1627752,-1.111940,203076,1.181818,203076,-0.236364
1,101108,201939,-3.521008,202691,-1.533333,1630228,1.770000,203952,-1.790000,202691,-0.242424,202691,-0.727273,1641764,0.980000,203952,0.100000
2,200768,1630178,-0.404762,202699,-1.047619,202699,0.722222,202699,-0.293651,1630178,-0.182540,202699,0.277778,1630178,-0.349206,202699,0.087302
3,201142,1626164,-6.208431,1628960,1.426056,1626220,-1.566667,201569,0.368486,203994,-0.311594,203994,-2.079710,203994,0.623188,1628960,0.197183
4,201143,1627759,-4.822368,1630573,1.396825,201950,-1.186404,204001,-0.215103,1628401,0.500000,1630573,-0.785714,204001,-1.147597,1627759,-0.089912
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239,1641739,1631200,2.700000,1629028,-2.415356,1631133,-0.868852,1631200,-0.570000,1631101,0.253929,1631200,0.490000,203924,-0.455128,1631101,-0.051282
240,1641764,201939,-5.784091,201939,-1.939394,101108,-1.261667,203110,-0.627755,203952,-0.290598,202691,-0.226087,201939,-1.159091,203952,-0.145299
241,1641787,1631093,4.733333,1631105,-1.528571,1630595,-0.597222,1631105,0.600000,1630595,-0.319444,1631093,0.466667,1631093,0.150000,1631105,0.000000
242,1641851,1629634,2.333333,1629634,-0.666667,1629634,0.000000,1629634,0.333333,1629634,0.000000,1629634,0.166667,1629634,0.000000,1629634,0.000000


In [46]:
if len(output) > 0:
    # Presuming 'server' and 'database' variables are defined elsewhere
    cnxn = pyodbc.connect(f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};')
    cursor = cnxn.cursor()
    
    cursor.execute("DELETE FROM InfluentialPlayersV1")
    cnxn.commit()

    # SQL INSERT statement for the 'InfluentialPlayersV1' table
    insert_stmt = '''
    INSERT INTO InfluentialPlayersV1 (
        PLAYER_ID, PTS_TEAMMATE_ID, PTS_IMPORTANCE_SCORE, 
        REB_TEAMMATE_ID, REB_IMPORTANCE_SCORE, 
        AST_TEAMMATE_ID, AST_IMPORTANCE_SCORE, 
        STL_TEAMMATE_ID, STL_IMPORTANCE_SCORE, 
        BLK_TEAMMATE_ID, BLK_IMPORTANCE_SCORE, 
        TOV_TEAMMATE_ID, TOV_IMPORTANCE_SCORE, 
        FG3M_TEAMMATE_ID, FG3M_IMPORTANCE_SCORE, 
        DD2_TEAMMATE_ID, DD2_IMPORTANCE_SCORE
    )
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    '''

    # Insert rows into the database table
    for index, row in output.iterrows():
        try:
            # Convert the relevant parts of the row to a tuple and execute the INSERT statement
            # Make sure the order of row's elements matches the order in the insert_stmt
            data_tuple = (
                row['PLAYER_ID'], row['PTS_TEAMMATE_ID'], row['PTS_IMPORTANCE_SCORE'], 
                row['REB_TEAMMATE_ID'], row['REB_IMPORTANCE_SCORE'], 
                row['AST_TEAMMATE_ID'], row['AST_IMPORTANCE_SCORE'], 
                row['STL_TEAMMATE_ID'], row['STL_IMPORTANCE_SCORE'], 
                row['BLK_TEAMMATE_ID'], row['BLK_IMPORTANCE_SCORE'], 
                row['TOV_TEAMMATE_ID'], row['TOV_IMPORTANCE_SCORE'], 
                row['FG3M_TEAMMATE_ID'], row['FG3M_IMPORTANCE_SCORE'], 
                row['DD2_TEAMMATE_ID'], row['DD2_IMPORTANCE_SCORE']
            )        
            cursor.execute(insert_stmt, data_tuple)
            cnxn.commit()
        except Exception as e:
            print(f"An error occurred at row {index}: {e}")
            # You can choose to either break or continue based on your preference

    cursor.close()
    cnxn.close()


In [47]:
server = 'localhost\SQLEXPRESS'
database = 'nba_game_data'

cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';')
cursor = cnxn.cursor()
sql = f"""
SELECT [SEASON_YEAR]
      ,[PLAYER_ID]
      ,[PLAYER_NAME]
      ,[NICKNAME]
      ,[TEAM_ID]
      ,[TEAM_ABBREVIATION]
      ,[TEAM_NAME]
      ,[GAME_ID]
      ,[GAME_DATE]
      ,avg(MIN) over (partition by PLAYER_ID order by GAME_DATE ROWS BETWEEN 82 PRECEDING AND CURRENT ROW) as Minutes_Per_Game
   
  FROM [nba_game_data].[dbo].[PlayerGameLogs]
  where yearSeason =2024
"""
df = pd.read_sql(sql,cnxn)

# Convert GAME_DATE to datetime and sort
df['GAME_DATE'] = pd.to_datetime(df['GAME_DATE'])
df = df.sort_values(by=['GAME_DATE'])

out_players_records = []

def was_player_traded(player_id, team_id, game_date):
    last_team_played_for = df[(df['PLAYER_ID'] == player_id) & (df['GAME_DATE'] < game_date)].tail(1)['TEAM_ID'].values
    return not last_team_played_for or (last_team_played_for[0] != team_id)

for game_id in df['GAME_ID'].unique():
    game_df = df[df['GAME_ID'] == game_id]
    teams_in_game = game_df['TEAM_ID'].unique()
    
    for team_id in teams_in_game:
        players_in_game = set(game_df[game_df['TEAM_ID'] == team_id]['PLAYER_ID'])
        previous_games = df[(df['TEAM_ID'] == team_id) & (df['GAME_DATE'] < game_df['GAME_DATE'].min())]
        expected_players = set(previous_games['PLAYER_ID'])
        
        out_players = []
        for player_id in expected_players:
            if player_id not in players_in_game and was_player_traded(player_id, team_id, game_df['GAME_DATE'].min()) == False:
                out_players.append(player_id)
        
        for pid in out_players:
            player_last_game = df[(df['PLAYER_ID'] == pid) & (df['GAME_DATE'] < game_df['GAME_DATE'].min())].tail(1)
            minutes_per_game = player_last_game['Minutes_Per_Game'].iloc[0] if not player_last_game.empty else 0
            player_name = df[df['PLAYER_ID'] == pid]['PLAYER_NAME'].iloc[0]
            
            out_players_records.append({
                'GAME_ID': game_id,
                'TEAM_ID': team_id,
                'OUT_PLAYER_ID': pid,
                'PLAYER_NAME': player_name,
                'Minutes_Per_Game': minutes_per_game
            })

out_players_df = pd.DataFrame(out_players_records)

# Calculating the counts
counts_df = out_players_df.groupby(['GAME_ID', 'TEAM_ID']).apply(
    lambda x: pd.Series({
        'number_of_players_who_over_avg_over_15_min_out': (x['Minutes_Per_Game'] > 15).sum(),
        'number_of_players_who_over_avg_over_20_min_out': (x['Minutes_Per_Game'] > 20).sum(),
        'number_of_players_who_over_avg_over_25_min_out': (x['Minutes_Per_Game'] > 25).sum()
    })
).reset_index()

cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';')
cursor = cnxn.cursor()

# Clear the table only if you're sure you want to remove existing data
cursor.execute("DELETE FROM player_injury_report")
cnxn.commit()

# Check if there are records to insert in the 'counts_df' DataFrame
if not counts_df.empty:
    insert_stmt = '''
    INSERT INTO player_injury_report (TEAM_ID, GAME_ID, number_of_players_who_over_avg_over_15_min_out, number_of_players_who_over_avg_over_20_min_out, number_of_players_who_over_avg_over_25_min_out)
    VALUES (?, ?, ?, ?, ?)
    '''

    # Insert rows into the database table
    for index, row in counts_df.iterrows():
        try:
            # Convert the relevant parts of the row to a tuple and execute the INSERT statement
            # Ensure the order of row's elements matches the order in the insert_stmt
            data_tuple = (
            int(row['TEAM_ID']),  # Convert TEAM_ID to Python int
            int(row['GAME_ID']),  # Convert GAME_ID to Python int
            int(row['number_of_players_who_over_avg_over_15_min_out']),
            int(row['number_of_players_who_over_avg_over_20_min_out']),
            int(row['number_of_players_who_over_avg_over_25_min_out'])
            )
            cursor.execute(insert_stmt, data_tuple)
            cnxn.commit()
        except Exception as e:
            print(f"An error occurred at row {index}: {e}")
            # Depending on your error handling preference, you might choose to break or continue

    cursor.close()
    cnxn.close()
else:
    print("No data to insert.")

## CommonTeamRoster

In [48]:
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';')
cursor = cnxn.cursor()
sql = "SELECT distinct [TEAM_ID] ,[yearSeason] FROM [nba_game_data].[dbo].[leagueGameLog] where yearSeason = {}".format(yearSeason)
all_teams = pd.read_sql(sql,cnxn)


In [49]:

# Initialize empty DataFrames for final results
final_roster = pd.DataFrame()
final_coach = pd.DataFrame()
try:
    # Loop through each row in the all_teams DataFrame
    for index, row in all_teams.iterrows():
        team_id = row['TEAM_ID']
        season = int(row['yearSeason'])

        # Fetching data for each team and season
        df = CommonTeamRoster(season=season-1, team_id=team_id)
        shot_data = json.loads(df.get_json())

        # Creating dataframes for each result set and appending to the final dataframes
        for result_set in shot_data['resultSets']:
            temp_df = pd.DataFrame(result_set['rowSet'], columns=result_set['headers'])

            # Assuming the first result set goes to final_df1 and the second to final_df2
            # Modify as needed based on the structure of your data
            if result_set['name'] == 'CommonTeamRoster':  # Replace 'SomeName1' with the actual name
                final_roster = pd.concat([final_roster, temp_df])
            elif result_set['name'] == 'Coaches':  # Replace 'SomeName2' with the actual name
                final_coach = pd.concat([final_coach, temp_df])

    # Resetting index of the final dataframes
    final_roster.reset_index(drop=True, inplace=True)
    final_coach.reset_index(drop=True, inplace=True)
except:
    print(f'some coach failed: {row}')

In [50]:
if len(final_roster)<= 0:
    pass
else:
    cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';')
    cursor = cnxn.cursor()

    # Example DELETE statement, modify as needed
    cursor.execute(F"DELETE FROM CommonTeamRoster WHERE [SEASON] = {yearSeason}")
    cnxn.commit()

    final_roster['SEASON'] = final_roster['SEASON'].astype(int)+1

    insert_stmt = '''
    INSERT INTO CommonTeamRoster (TeamID, SEASON, LeagueID, PLAYER, NICKNAME, PLAYER_SLUG, NUM, POSITION, HEIGHT, WEIGHT, BIRTH_DATE, AGE, EXP, SCHOOL, PLAYER_ID, HOW_ACQUIRED)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    '''
    # Assuming you have a way to transform or map your DataFrame columns to the table columns
    for index, row in final_roster.iterrows():
        try:
            # Map or transform 'row' to match the columns of CommonTeamRoster
            transformed_row = row  # Implement this function based on your needs
            cursor.execute(insert_stmt, tuple(transformed_row))
            cnxn.commit()
        except Exception as e:
            print(f"An error occurred at row {index}: {e}")
            # Decide whether to break or continue


    cursor.close()
    cnxn.close()


In [51]:
if len(final_coach)<= 0:
    pass
else:
    cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';')
    cursor = cnxn.cursor()

    # Example DELETE statement, adjust as needed
    cursor.execute(f"DELETE FROM NBACoachs WHERE [SEASON] = {yearSeason}")
    cnxn.commit()

    insert_stmt = '''
    INSERT INTO NBACoachs 
    (TEAM_ID, SEASON, COACH_ID, FIRST_NAME, LAST_NAME, COACH_NAME, IS_ASSISTANT, COACH_TYPE, SORT_SEQUENCE, SUB_SORT_SEQUENCE)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)

    '''

    for index, row in final_coach.iterrows():
        try:

            cursor.execute(insert_stmt, tuple(row))
            cnxn.commit()
        except Exception as e:
            pass

    cursor.close()
    cnxn.close()


In [52]:
del final_roster, final_coach, all_teams, df, shot_data, temp_df

# Defense 

In [53]:
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';')
cursor = cnxn.cursor()
sql = "SELECT distinct trim(cast(FORMAT(GAME_ID, '0000000000') as char)) AS GAME_ID FROM [nba_game_data].[dbo].[leagueGameLog] where yearSeason >= 2018 --0{}"#.format(yearSeason)
all_games = pd.read_sql(sql,cnxn)
sql = "SELECT distinct gameId AS GAME_ID FROM [nba_game_data].[dbo].[BoxScoreDefensiveV2]"#.format(yearSeason)
all_defense = pd.read_sql(sql,cnxn)

In [54]:
missing_games = all_games.merge(all_defense, on=['GAME_ID'],how='left', indicator=True)
missing_games = missing_games.loc[missing_games['_merge']=='left_only'].reset_index(drop=True)
missing_games['GAME_ID'] = missing_games['GAME_ID'].astype(str).str.zfill(10)
missing_games

,GAME_ID,_merge
0,0021800911,left_only
1,0022301178,left_only
2,0021800665,left_only
3,0022301193,left_only
4,0022301190,left_only
5,0022301194,left_only
6,0022301196,left_only
7,0022301200,left_only
8,0021800014,left_only
9,0021900792,left_only


In [55]:
missing_games = missing_games['GAME_ID'].to_list()

In [56]:
final_defense = pd.DataFrame()
i = 1
for game_id in missing_games:
    if i% 250 ==0:
        print(i)
    i += 1
    try:
        box_defense = BoxScoreDefensiveV2(game_id=game_id)

        # Fetch the data and convert it to a DataFrame
        data_frames = box_defense.get_data_frames()

        # Assuming you want the first DataFrame
        df = data_frames[0] if data_frames else pd.DataFrame()
        final_defense = pd.concat([final_defense,df]).reset_index(drop=True)
    except:
        pass
    

In [57]:
# Function to convert time format to total minutes
def convert_to_decimal_minutes(time_str):
    minutes, seconds = map(int, time_str.split(':'))
    return minutes + seconds / 60



In [58]:
if len(final_defense)>0:
    final_defense['matchupMinutes'] = final_defense['matchupMinutes'].apply(convert_to_decimal_minutes)
    cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER=' + server + ';DATABASE=' + database + ';')
    cursor = cnxn.cursor()

    # SQL insert statement with updated column names
    insert_stmt = '''
    INSERT INTO BoxScoreDefensiveV2 (
      gameId, teamId, teamCity, teamName, teamTricode, teamSlug, personId, firstName, familyName, 
      nameI, playerSlug, position, comment, jerseyNum, matchupMinutes, partialPossessions, 
      switchesOn, playerPoints, defensiveRebounds, matchupAssists, matchupTurnovers, steals, 
      blocks, matchupFieldGoalsMade, matchupFieldGoalsAttempted, matchupFieldGoalPercentage, 
      matchupThreePointersMade, matchupThreePointersAttempted, matchupThreePointerPercentage
    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    '''

    # Loop through DataFrame rows and insert into database
    for index, row in final_defense.iterrows():
        try:
    #         Execute the insert statement
            cursor.execute(insert_stmt, tuple(row))
            cnxn.commit()
        except Exception as e:
            print(f"An error occurred at row {index}: {e}")
    #         Depending on your preference, you can break, continue, or retry

    # Close the connection
    cursor.close()
    cnxn.close()
    del final_defense, df, all_defense,missing_games

In [59]:
# del final_defense, df, all_defense,missing_games

## BoxScoreMatchupsV3

In [60]:
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';')
cursor = cnxn.cursor()
sql = "SELECT distinct trim(cast(FORMAT(GAME_ID, '0000000000') as char)) AS GAME_ID FROM [nba_game_data].[dbo].[leagueGameLog] where yearSeason >= 2018 --0{}"#.format(yearSeason)
all_games = pd.read_sql(sql,cnxn)
sql = "SELECT distinct gameId AS GAME_ID FROM [nba_game_data].[dbo].[BoxScoreMatchupsV3]"#.format(yearSeason)
all_match = pd.read_sql(sql,cnxn)

In [61]:
missing_games = all_games.merge(all_match, on=['GAME_ID'],how='left', indicator=True)
missing_games = missing_games.loc[missing_games['_merge']=='left_only'].reset_index(drop=True)
missing_games['GAME_ID'] = missing_games['GAME_ID'].astype(str).str.zfill(10)
missing_games

,GAME_ID,_merge
0,0021800911,left_only
1,0022301178,left_only
2,0021800665,left_only
3,0022301193,left_only
4,0022301190,left_only
5,0022301194,left_only
6,0022301196,left_only
7,0022301200,left_only
8,0021800014,left_only
9,0021900792,left_only


In [62]:
missing_games = missing_games['GAME_ID'].to_list()

In [63]:
final_match = pd.DataFrame()
i = 1
for game_id_one in missing_games:
    if i% 250 ==0:
        print(i)
    i += 1
    try:
        data_frames = BoxScoreMatchupsV3(game_id=game_id_one).get_data_frames()
        df = data_frames[0] if data_frames else pd.DataFrame()
        df['matchupMinutes'] = df['matchupMinutes'].apply(convert_to_decimal_minutes)

        final_match = pd.concat([final_match,df]).reset_index(drop=True)
    except:
        pass

In [64]:
if len(final_match)> 0:
    cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER=' + server + ';DATABASE=' + database + ';')
    cursor = cnxn.cursor()

    # SQL insert statement with updated column names
    insert_stmt = '''
    INSERT INTO BoxScoreMatchupsV3 (
      [gameId], [teamId], [teamCity], [teamName], [teamTricode], [teamSlug], [personIdOff], 
      [firstNameOff], [familyNameOff], [nameIOff], [playerSlugOff], [positionOff], [commentOff], 
      [jerseyNumOff], [personIdDef], [firstNameDef], [familyNameDef], [nameIDef], [playerSlugDef], 
      [jerseyNumDef], [matchupMinutes], [matchupMinutesSort], [partialPossessions], 
      [percentageDefenderTotalTime], [percentageOffensiveTotalTime], [percentageTotalTimeBothOn], 
      [switchesOn], [playerPoints], [teamPoints], [matchupAssists], [matchupPotentialAssists], 
      [matchupTurnovers], [matchupBlocks], [matchupFieldGoalsMade], [matchupFieldGoalsAttempted], 
      [matchupFieldGoalsPercentage], [matchupThreePointersMade], [matchupThreePointersAttempted], 
      [matchupThreePointersPercentage], [helpBlocks], [helpFieldGoalsMade], [helpFieldGoalsAttempted], 
      [helpFieldGoalsPercentage], [matchupFreeThrowsMade], [matchupFreeThrowsAttempted], [shootingFouls]
    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    '''

    # Loop through DataFrame rows and insert into the database
    for index, row in final_match.iterrows():
        try:
            # Execute the insert statement
            cursor.execute(insert_stmt, tuple(row))
            cnxn.commit()
        except Exception as e:
            print(f"An error occurred at row {index}: {e}")

            # Depending on your preference, you can break, continue, or retry

    # Close the connection
    cursor.close()
    cnxn.close()
    del all_match, final_match, missing_games, df


In [65]:
# del all_match, final_match, missing_games, df

## BoxScoreAdvancedV3

In [66]:
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';')
cursor = cnxn.cursor()
sql = "SELECT distinct cast(FORMAT(GAME_ID, '0000000000') as char) as GAME_ID FROM [nba_game_data].[dbo].[leagueGameLog] --where yearSeason = {}"#.format(yearSeason)
all_games = pd.read_sql(sql,cnxn)
# all_games

In [67]:
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';')
cursor = cnxn.cursor()
sql = "SELECT distinct cast(FORMAT([GAME_ID], '0000000000') as char) as GAME_ID FROM [nba_game_data].[dbo].[BoxScoreAdvancedV3] --where yearSeason = {}"#.format(yearSeason)
data = pd.read_sql(sql,cnxn)
cnxn.close()


In [68]:
# cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';')
# cursor = cnxn.cursor()
# sql = "SELECT distinct FORMAT(cast(gameId as char), '0000000000') as GAME_ID FROM [nba_game_data].[dbo].[TeamBoxScoreAdvancedV3]"#.format(yearSeason)
# data = pd.read_sql(sql,cnxn)
# cnxn.close()

In [69]:
advanced = all_games.merge(data, on=['GAME_ID'],how='left', indicator=True)
advanced = advanced.loc[advanced['_merge']=='left_only'].reset_index(drop=True)
advanced['GAME_ID'] = advanced['GAME_ID'].astype(str).str.zfill(10)
advanced

,GAME_ID,_merge
0,0022301193,left_only
1,0022301194,left_only
2,0022301190,left_only
3,0022301196,left_only
4,0022301200,left_only
5,0022301195,left_only
6,0022301187,left_only
7,0022301188,left_only
8,0022301197,left_only
9,0022301198,left_only


In [70]:
advanced_stats = pd.DataFrame()
advanced_team_stats = pd.DataFrame()
tracker = 0 
for index, row in advanced.iterrows():
    tracker +=1
    if index%500 == 0 and index!=0:
        print(index)
    if index%5000 == 0 and index!=0:
        print(index)
    try:
        df = BoxScoreAdvancedV3(game_id = row['GAME_ID'].replace(" ",""),timeout=100).get_data_frames()
        team = df[1]
        df = df[0]
        # Create a DataFrame from the flattened data
        df['GAME_ID'] = row['GAME_ID']
        advanced_stats = pd.concat([advanced_stats,df]).reset_index(drop=True)
        advanced_team_stats =  pd.concat([advanced_team_stats,team]).reset_index(drop=True)
    except:
        pass


In [71]:
advanced_stats['teamType'] = ''

In [72]:
if len(advanced_stats) > 0:
    cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';')
    cursor = cnxn.cursor()

    # Rearrange DataFrame columns to match the SQL table structure
    columns_order = ['GAME_ID','teamId', 'teamName', 'personId', 'firstName', 'familyName', 'position', 'teamType', 'minutes', 'estimatedOffensiveRating', 'offensiveRating', 'estimatedDefensiveRating', 'defensiveRating', 'estimatedNetRating', 'netRating', 'assistPercentage', 'assistToTurnover', 'assistRatio', 'offensiveReboundPercentage', 'defensiveReboundPercentage', 'reboundPercentage', 'turnoverRatio', 'effectiveFieldGoalPercentage', 'trueShootingPercentage', 'usagePercentage', 'estimatedUsagePercentage', 'estimatedPace', 'pace', 'pacePer40', 'possessions', 'PIE']
    advanced_stats = advanced_stats[columns_order]

    # Prepare the INSERT SQL statement
    insert_stmt = '''
    INSERT INTO BoxScoreAdvancedV3 (GAME_ID,teamId, teamName, personId, firstName, familyName, position, teamType, minutes, estimatedOffensiveRating, offensiveRating, estimatedDefensiveRating, defensiveRating, estimatedNetRating, netRating, assistPercentage, assistToTurnover, assistRatio, offensiveReboundPercentage, defensiveReboundPercentage, reboundPercentage, turnoverRatio, effectiveFieldGoalPercentage, trueShootingPercentage, usagePercentage, estimatedUsagePercentage, estimatedPace, pace, pacePer40, possessions, PIE)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    '''

    # Iterate over DataFrame rows
    for index, row in advanced_stats.iterrows():
        try:
            # Execute the INSERT statement for each row
            cursor.execute(insert_stmt, tuple(row))
            cnxn.commit()
        except Exception as e:
            print(f"An error occurred at row {index}: {e}")
            # Optionally, break or continue based on the nature of the error

    # Close the connection
    cursor.close()
    cnxn.close()

In [73]:
if len(advanced_team_stats) > 0:
    # Database connection
    cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER=' + server + ';DATABASE=' + database + ';')
    cursor = cnxn.cursor()

    # Rearrange DataFrame columns to match the SQL table structure
    columns_order = ['gameId', 'teamId', 'teamCity', 'teamName', 'teamTricode', 'teamSlug', 'minutes', 'estimatedOffensiveRating', 'offensiveRating', 'estimatedDefensiveRating', 'defensiveRating', 'estimatedNetRating', 'netRating', 'assistPercentage', 'assistToTurnover', 'assistRatio', 'offensiveReboundPercentage', 'defensiveReboundPercentage', 'reboundPercentage', 'estimatedTeamTurnoverPercentage', 'turnoverRatio', 'effectiveFieldGoalPercentage', 'trueShootingPercentage', 'usagePercentage', 'estimatedUsagePercentage', 'estimatedPace', 'pace', 'pacePer40', 'possessions', 'PIE']
    advanced_team_stats = advanced_team_stats[columns_order]

    # Prepare the INSERT SQL statement
    insert_stmt = '''
    INSERT INTO TeamBoxScoreAdvancedV3 (gameId, teamId, teamCity, teamName, teamTricode, teamSlug, minutes, estimatedOffensiveRating, offensiveRating, estimatedDefensiveRating, defensiveRating, estimatedNetRating, netRating, assistPercentage, assistToTurnover, assistRatio, offensiveReboundPercentage, defensiveReboundPercentage, reboundPercentage, estimatedTeamTurnoverPercentage, turnoverRatio, effectiveFieldGoalPercentage, trueShootingPercentage, usagePercentage, estimatedUsagePercentage, estimatedPace, pace, pacePer40, possessions, PIE)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    '''

    # Iterate over DataFrame rows and insert data
    for index, row in advanced_team_stats.iterrows():
        try:
            cursor.execute(insert_stmt, tuple(row))
            cnxn.commit()
        except Exception as e:
            print(f"An error occurred at row {index}: {e}")
            # Handle the error based on your requirements (break, continue, etc.)

    # Close the connection
    cursor.close()
    cnxn.close()
    del advanced_team_stats, advanced_stats, team, all_games, data


In [74]:
# del advanced_team_stats, advanced_stats, team, all_games, data

In [77]:
# server = 'localhost\SQLEXPRESS'
# database = 'nba_game_data'

# cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';')
# cursor = cnxn.cursor()
# sql = f"""
# SELECT [gameId]
# 	  , lgl.GAME_DATE
#       ,[teamId]
#       ,[teamCity]
#       ,[teamName]
#       ,[teamTricode]
#       ,[teamSlug]
#       ,[minutes]
#       ,[estimatedOffensiveRating]
#       ,[offensiveRating]
#       ,[estimatedDefensiveRating]
#       ,[defensiveRating]
#       ,[estimatedNetRating]
#       ,[netRating]
#       ,[assistPercentage]
#       ,[assistToTurnover]
#       ,[assistRatio]
#       ,[offensiveReboundPercentage]
#       ,[defensiveReboundPercentage]
#       ,[reboundPercentage]
#       ,[estimatedTeamTurnoverPercentage]
#       ,[turnoverRatio]
#       ,[effectiveFieldGoalPercentage]
#       ,[trueShootingPercentage]
#       ,[usagePercentage]
#       ,[estimatedUsagePercentage]
#       ,[estimatedPace]
#       ,[pace]
#       ,[pacePer40]
#       ,[possessions]
#       ,[PIE]
#   FROM [nba_game_data].[dbo].[TeamBoxScoreAdvancedV3] tbsa
#     left outer join [nba_game_data].[dbo].[LeagueGameLog] lgl
#   on 
#   tbsa.teamId = lgl.TEAM_ID
#   and tbsa.gameId = lgl.GAME_ID
# where lgl.yearSeason = 2024
# """
# df = pd.read_sql(sql,cnxn)

# # Convert the 'GAME_DATE' column to datetime if not already
# df['GAME_DATE'] = pd.to_datetime(df['GAME_DATE'])

# # Ensure the data is sorted
# df.sort_values(by=['teamId', 'GAME_DATE'], inplace=True)
# test = df.iloc[-1]
# test['GAME_DATE'] = pd.Timestamp.today()
# df = pd.concat([df,pd.DataFrame([test])])

# # Create a unique list of all teams and all dates
# unique_teams = df['teamId'].unique()
# unique_dates = pd.date_range(start=df['GAME_DATE'].min(), end=df['GAME_DATE'].max())

# # Create a DataFrame from the product of unique_dates and unique_teams
# from itertools import product
# all_combinations = pd.DataFrame(product(unique_dates, unique_teams), columns=['GAME_DATE', 'teamId'])

# # Calculate the running average for defensiveRating, excluding the current game
# df['cumulativeDR'] = df.groupby('teamId')['defensiveRating'].apply(lambda x: x.shift(1).expanding().mean())

# # Merge the running average back onto the all_combinations DataFrame
# complete_df = pd.merge(all_combinations, df[['GAME_DATE', 'teamId', 'cumulativeDR']], on=['GAME_DATE', 'teamId'], how='left')

# # Forward fill the missing values for each team to carry the last known average forward
# complete_df['cumulativeDR'] = complete_df.groupby('teamId')['cumulativeDR'].ffill()

# # First, ensure the DataFrame is sorted by date and then by the defensive rating in ascending order 
# # (since a lower defensive rating is better in basketball).
# complete_df.sort_values(by=['GAME_DATE', 'cumulativeDR'], ascending=[True, True], inplace=True)

# # Rank the teams within each day
# complete_df['daily_rank'] = complete_df.groupby('GAME_DATE')['cumulativeDR'].rank(method='min')

# complete_df=complete_df.dropna()
# # complete_df.sort_values(by='GAME_DATE')
# import pyodbc

# # Assuming 'server' and 'database' variables are defined elsewhere
# cnxn = pyodbc.connect(f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};')
# cursor = cnxn.cursor()

# # Clear existing data from the table if necessary
# # Be very careful with this step, as it will remove all existing data
# cursor.execute("DELETE FROM teamDailyRankings")
# cnxn.commit()

# # Check if there are records to insert
# if len(complete_df) > 0:
#     # SQL INSERT statement for the 'teamDailyRankings' table
#     insert_stmt = '''
#     INSERT INTO teamDailyRankings (GAME_DATE, teamId, cumulativeDR, daily_rank)
#     VALUES (?, ?, ?, ?)
#     '''

#     # Insert rows into the database table
#     for index, row in complete_df.iterrows():
#         try:
#             # Convert the relevant parts of the row to a tuple and execute the INSERT statement
#             data_tuple = (
#                 row['GAME_DATE'].to_pydatetime(),  # Convert to datetime object if not already
#                 int(row['teamId']),
#                 float(row['cumulativeDR']),  # Ensure this is a float
#                 float(row['daily_rank'])  # Ensure this is a float
#             )
#             cursor.execute(insert_stmt, data_tuple)
#             # You might want to commit less frequently if you're inserting a lot of rows to improve performance
#         except Exception as e:
#             print(f"An error occurred at row {index}: {e}")
#             # Depending on your preference for error handling, break or continue

#     cnxn.commit()  # Committing the transaction after all rows have been inserted
#     cursor.close()
#     cnxn.close()
# else:
#     print("No data to insert.")

# BoxScoreHustleV2

In [78]:
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';')
cursor = cnxn.cursor()
sql = "SELECT distinct cast(FORMAT(GAME_ID, '0000000000') as char) as GAME_ID FROM [nba_game_data].[dbo].[leagueGameLog] where yearSeason >2016"#.format(yearSeason)
all_games = pd.read_sql(sql,cnxn)
# all_games

In [79]:
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';')
cursor = cnxn.cursor()
sql = "SELECT  DISTINCT CAST(FORMAT(CAST([gameId] AS BIGINT), '0000000000') AS CHAR) AS GAME_ID FROM [nba_game_data].[dbo].[BoxScoreHustleV2]"
data = pd.read_sql(sql,cnxn)
cnxn.close()

In [80]:
hustle = all_games.merge(data, on=['GAME_ID'],how='left', indicator=True)
hustle = hustle.loc[hustle['_merge']=='left_only'].reset_index(drop=True)
hustle['GAME_ID'] = hustle['GAME_ID'].astype(str).str.zfill(10)
hustle

,GAME_ID,_merge
0,0022301193,left_only
1,0022301190,left_only
2,0022301194,left_only
3,0022301196,left_only
4,0022301200,left_only
5,0022301195,left_only
6,0022301187,left_only
7,0022301188,left_only
8,0022301197,left_only
9,0022301198,left_only


In [81]:
hustle_stats = pd.DataFrame()
hustle_team_stats = pd.DataFrame()

for index, row in hustle.iterrows():
    if index%500 == 0 and index!=0:
        print(index)
    if index%5000 == 0 and index!=0:
        print(index)
    try:
        flattened_data = []

        df = BoxScoreHustleV2(game_id = row['GAME_ID'].replace(" ",""),timeout=100).get_data_frames()
        team = df[1]
        df=df[0]
#         Create a DataFrame from the flattened data
        df['GAME_ID'] = row['GAME_ID']
        hustle_stats = pd.concat([hustle_stats,df]).reset_index(drop=True)
        hustle_team_stats = pd.concat([hustle_team_stats,team]).reset_index(drop=True)

    except:
        pass

In [82]:
if len(hustle_stats) <=0:
    pass 
else:
    cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';')
    cursor = cnxn.cursor()

    # Rearrange DataFrame columns to match the BoxScoreHustleV2 table structure
    columns_order = ['gameId', 'teamId', 'teamCity', 'teamName', 'teamTricode', 'teamSlug', 'personId', 'firstName', 'familyName', 'nameI', 'playerSlug', 'position', 'comment', 'jerseyNum', 'minutes', 'points', 'contestedShots', 'contestedShots2pt', 'contestedShots3pt', 'deflections', 'chargesDrawn', 'screenAssists', 'screenAssistPoints', 'looseBallsRecoveredOffensive', 'looseBallsRecoveredDefensive', 'looseBallsRecoveredTotal', 'offensiveBoxOuts', 'defensiveBoxOuts', 'boxOutPlayerTeamRebounds', 'boxOutPlayerRebounds', 'boxOuts']
    hustle_stats = hustle_stats[columns_order]  # Replace 'advanced_stats' with your DataFrame name

    # Prepare the INSERT SQL statement for BoxScoreHustleV2
    insert_stmt = '''
    INSERT INTO BoxScoreHustleV2 (gameId, teamId, teamCity, teamName, teamTricode, teamSlug, personId, firstName, familyName, nameI, playerSlug, position, comment, jerseyNum, minutes, points, contestedShots, contestedShots2pt, contestedShots3pt, deflections, chargesDrawn, screenAssists, screenAssistPoints, looseBallsRecoveredOffensive, looseBallsRecoveredDefensive, looseBallsRecoveredTotal, offensiveBoxOuts, defensiveBoxOuts, boxOutPlayerTeamRebounds, boxOutPlayerRebounds, boxOuts)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    '''

    # Iterate over DataFrame rows and insert data
    for index, row in hustle_stats.iterrows():
        try:
            cursor.execute(insert_stmt, tuple(row))
            cnxn.commit()
        except Exception as e:
            print(f"An error occurred at row {index}: {e}")

    # Close the connection
    cursor.close()
    cnxn.close()


In [83]:
# Check if the DataFrame is empty
if len(hustle_team_stats) > 0:
    # Database connection
    cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER=' + server + ';DATABASE=' + database + ';')
    cursor = cnxn.cursor()

    # Rearrange DataFrame columns to match the SQL table structure
    columns_order = ['gameId', 'teamId', 'teamCity', 'teamName', 'teamTricode', 'teamSlug', 'minutes', 'points', 'contestedShots', 'contestedShots2pt', 'contestedShots3pt', 'deflections', 'chargesDrawn', 'screenAssists', 'screenAssistPoints', 'looseBallsRecoveredOffensive', 'looseBallsRecoveredDefensive', 'looseBallsRecoveredTotal', 'offensiveBoxOuts', 'defensiveBoxOuts', 'boxOutPlayerTeamRebounds', 'boxOutPlayerRebounds', 'boxOuts']
    hustle_team_stats = hustle_team_stats[columns_order]

    # Prepare the INSERT SQL statement
    insert_stmt = '''
    INSERT INTO TeamBoxScoreHustleV2 (gameId, teamId, teamCity, teamName, teamTricode, teamSlug, minutes, points, contestedShots, contestedShots2pt, contestedShots3pt, deflections, chargesDrawn, screenAssists, screenAssistPoints, looseBallsRecoveredOffensive, looseBallsRecoveredDefensive, looseBallsRecoveredTotal, offensiveBoxOuts, defensiveBoxOuts, boxOutPlayerTeamRebounds, boxOutPlayerRebounds, boxOuts)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    '''

    # Iterate over DataFrame rows and insert data
    for index, row in hustle_team_stats.iterrows():
        try:
            cursor.execute(insert_stmt, tuple(row))
            cnxn.commit()
        except Exception as e:
            print(f"An error occurred at row {index}: {e}")
            # Handle the error based on your requirements (break, continue, etc.)

    # Close the connection
    cursor.close()
    cnxn.close()


In [84]:
del hustle_team_stats, hustle_stats

# BoxScoreMiscV3

In [85]:
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';')
cursor = cnxn.cursor()
sql = "SELECT distinct cast(FORMAT(GAME_ID, '0000000000') as char) as GAME_ID FROM [nba_game_data].[dbo].[leagueGameLog] --where yearSeason = {}"#.format(yearSeason)
all_games = pd.read_sql(sql,cnxn)
# all_games

In [86]:
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';')
cursor = cnxn.cursor()
sql = "SELECT  DISTINCT CAST(FORMAT(CAST([gameId] AS BIGINT), '0000000000') AS CHAR) AS GAME_ID FROM [nba_game_data].[dbo].[BoxScoreMiscV3]"
data = pd.read_sql(sql,cnxn)
cnxn.close()

In [87]:
misc = all_games.merge(data, on=['GAME_ID'],how='left', indicator=True)
misc = misc.loc[misc['_merge']=='left_only'].reset_index(drop=True)
misc['GAME_ID'] = misc['GAME_ID'].astype(str).str.zfill(10)
misc

,GAME_ID,_merge
0,0022301193,left_only
1,0022301194,left_only
2,0022301190,left_only
3,0022301196,left_only
4,0022301200,left_only
5,0022301195,left_only
6,0022301187,left_only
7,0022301188,left_only
8,0022301197,left_only
9,0022301198,left_only


In [88]:
misc_stats = pd.DataFrame()
team_misc_stats = pd.DataFrame()

for index, row in misc.iterrows():
    if index%500 == 0 and index!=0:
        print(index)
    if index%5000 == 0 and index!=0:
        print(index)
    try:
        flattened_data = []

        df = BoxScoreMiscV3(game_id = row['GAME_ID'].replace(" ",""),timeout=100).get_data_frames()
        team = df[1]
        df = df[0]
        # Create a DataFrame from the flattened data
        df['GAME_ID'] = row['GAME_ID']
        misc_stats = pd.concat([misc_stats,df]).reset_index(drop=True)
        team_misc_stats = pd.concat([team_misc_stats,team]).reset_index(drop=True)
        
    except:
        pass


In [89]:
if len(misc_stats) <=0:
    pass 
else:
    cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';')
    cursor = cnxn.cursor()

    # Rearrange DataFrame columns to match the BoxScoreMiscV3 table structure
    columns_order = ['gameId', 'teamId', 'teamCity', 'teamName', 'teamTricode', 'teamSlug', 'personId', 'firstName', 'familyName', 'nameI', 'playerSlug', 'position', 'comment', 'jerseyNum', 'minutes', 'pointsOffTurnovers', 'pointsSecondChance', 'pointsFastBreak', 'pointsPaint', 'oppPointsOffTurnovers', 'oppPointsSecondChance', 'oppPointsFastBreak', 'oppPointsPaint', 'blocks', 'blocksAgainst', 'foulsPersonal', 'foulsDrawn']
    misc_stats = misc_stats[columns_order]  # Replace 'advanced_stats' with your DataFrame name

    # Prepare the INSERT SQL statement for BoxScoreMiscV3
    insert_stmt = '''
    INSERT INTO BoxScoreMiscV3 (gameId, teamId, teamCity, teamName, teamTricode, teamSlug, personId, firstName, familyName, nameI, playerSlug, position, comment, jerseyNum, minutes, pointsOffTurnovers, pointsSecondChance, pointsFastBreak, pointsPaint, oppPointsOffTurnovers, oppPointsSecondChance, oppPointsFastBreak, oppPointsPaint, blocks, blocksAgainst, foulsPersonal, foulsDrawn)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    '''

    # Iterate over DataFrame rows and insert data
    for index, row in misc_stats.iterrows():
        try:
            cursor.execute(insert_stmt, tuple(row))
            cnxn.commit()
        except Exception as e:
            print(f"An error occurred at row {index}: {e}")

    # Close the connection
    cursor.close()
    cnxn.close()


In [90]:
if len(team_misc_stats) > 0:
    # Database connection
    cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER=' + server + ';DATABASE=' + database + ';')
    cursor = cnxn.cursor()

    # Rearrange DataFrame columns to match the SQL table structure
    columns_order = ['gameId', 'teamId', 'teamCity', 'teamName', 'teamTricode', 'teamSlug', 'minutes', 'pointsOffTurnovers', 'pointsSecondChance', 'pointsFastBreak', 'pointsPaint', 'oppPointsOffTurnovers', 'oppPointsSecondChance', 'oppPointsFastBreak', 'oppPointsPaint', 'blocks', 'blocksAgainst', 'foulsPersonal', 'foulsDrawn']
    team_misc_stats = team_misc_stats[columns_order]

    # Prepare the INSERT SQL statement
    insert_stmt = '''
    INSERT INTO TeamBoxScoreMiscV3 (gameId, teamId, teamCity, teamName, teamTricode, teamSlug, minutes, pointsOffTurnovers, pointsSecondChance, pointsFastBreak, pointsPaint, oppPointsOffTurnovers, oppPointsSecondChance, oppPointsFastBreak, oppPointsPaint, blocks, blocksAgainst, foulsPersonal, foulsDrawn)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    '''

    # Iterate over DataFrame rows and insert data
    for index, row in team_misc_stats.iterrows():
        try:
            cursor.execute(insert_stmt, tuple(row))
            cnxn.commit()
        except Exception as e:
            print(f"An error occurred at row {index}: {e}")
            # Handle the error based on your requirements (break, continue, etc.)

    # Close the connection
    cursor.close()
    cnxn.close()
    del team_misc_stats, misc_stats, df, team

# BoxScorePlayerTrackV3

In [91]:
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';')
cursor = cnxn.cursor()
sql = "SELECT distinct cast(FORMAT(GAME_ID, '0000000000') as char) as GAME_ID FROM [nba_game_data].[dbo].[leagueGameLog] --where yearSeason = {}"#.format(yearSeason)
all_games = pd.read_sql(sql,cnxn)
# all_games

In [92]:
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';')
cursor = cnxn.cursor()
sql = "SELECT  DISTINCT CAST(FORMAT(CAST([gameId] AS BIGINT), '0000000000') AS CHAR) AS GAME_ID FROM [nba_game_data].[dbo].[BoxScorePlayerTrackV3]"
data = pd.read_sql(sql,cnxn)
cnxn.close()

In [93]:
track = all_games.merge(data, on=['GAME_ID'],how='left', indicator=True)
track = track.loc[track['_merge']=='left_only'].reset_index(drop=True)
track['GAME_ID'] = track['GAME_ID'].astype(str).str.zfill(10)
track

,GAME_ID,_merge
0,0022301193,left_only
1,0022301194,left_only
2,0022301190,left_only
3,0022301196,left_only
4,0022301200,left_only
5,0022301195,left_only
6,0022301187,left_only
7,0022301188,left_only
8,0022301197,left_only
9,0022301198,left_only


In [94]:
track_stats = pd.DataFrame()
team_track_stats = pd.DataFrame()

for index, row in track.iterrows():
    if index%500 == 0 and index!=0:
        print(index)
    if index%5000 == 0 and index!=0:
        print(index)
    try:
        flattened_data = []

        df = BoxScorePlayerTrackV3(game_id = row['GAME_ID'].replace(" ",""),timeout=100).get_data_frames()
        team = df[1]
        df=df[0]
#         Create a DataFrame from the flattened data
        df['GAME_ID'] = row['GAME_ID']
        track_stats = pd.concat([track_stats,df]).reset_index(drop=True)
        team_track_stats = pd.concat([team_track_stats,team]).reset_index(drop=True)
        
        
    except:
        pass


In [95]:
if len(track_stats) <=0:
    pass 
else:
    cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';')
    cursor = cnxn.cursor()

    # Rearrange DataFrame columns to match the new structure
    new_columns_order = [
        'gameId', 'teamId', 'teamCity', 'teamName', 'teamTricode', 'teamSlug',
        'personId', 'firstName', 'familyName', 'nameI', 'playerSlug', 'position',
        'comment', 'jerseyNum', 'minutes', 'speed', 'distance',
        'reboundChancesOffensive', 'reboundChancesDefensive', 'reboundChancesTotal',
        'touches', 'secondaryAssists', 'freeThrowAssists', 'passes', 'assists',
        'contestedFieldGoalsMade', 'contestedFieldGoalsAttempted', 'contestedFieldGoalPercentage',
        'uncontestedFieldGoalsMade', 'uncontestedFieldGoalsAttempted', 'uncontestedFieldGoalsPercentage',
        'fieldGoalPercentage', 'defendedAtRimFieldGoalsMade', 'defendedAtRimFieldGoalsAttempted',
        'defendedAtRimFieldGoalPercentage'
    ]

    track_stats = track_stats[new_columns_order]  # Replace 'track_stats' with your DataFrame name

    # Prepare the INSERT SQL statement for the new table structure
    insert_stmt = '''
    INSERT INTO BoxScorePlayerTrackV3 (
        gameId, teamId, teamCity, teamName, teamTricode, teamSlug, personId, firstName,
        familyName, nameI, playerSlug, position, comment, jerseyNum, minutes, speed,
        distance, reboundChancesOffensive, reboundChancesDefensive, reboundChancesTotal,
        touches, secondaryAssists, freeThrowAssists, passes, assists, contestedFieldGoalsMade,
        contestedFieldGoalsAttempted, contestedFieldGoalPercentage, uncontestedFieldGoalsMade,
        uncontestedFieldGoalsAttempted, uncontestedFieldGoalsPercentage, fieldGoalPercentage,
        defendedAtRimFieldGoalsMade, defendedAtRimFieldGoalsAttempted, defendedAtRimFieldGoalPercentage
    )
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    '''

    # Iterate over DataFrame rows and insert data
    for index, row in track_stats.iterrows():
        try:
            cursor.execute(insert_stmt, tuple(row))
            cnxn.commit()
        except Exception as e:
            print(f"An error occurred at row {index}: {e}")

    # Close the connection
    cursor.close()
    cnxn.close()

In [96]:
if len(team_track_stats) > 0:
    # Database connection
    cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER=' + server + ';DATABASE=' + database + ';')
    cursor = cnxn.cursor()

    # Rearrange DataFrame columns to match the SQL table structure
    columns_order = ['gameId', 'teamId', 'teamCity', 'teamName', 'teamTricode', 'teamSlug', 'minutes', 'distance', 'reboundChancesOffensive', 'reboundChancesDefensive', 'reboundChancesTotal', 'touches', 'secondaryAssists', 'freeThrowAssists', 'passes', 'assists', 'contestedFieldGoalsMade', 'contestedFieldGoalsAttempted', 'contestedFieldGoalPercentage', 'uncontestedFieldGoalsMade', 'uncontestedFieldGoalsAttempted', 'uncontestedFieldGoalsPercentage', 'fieldGoalPercentage', 'defendedAtRimFieldGoalsMade', 'defendedAtRimFieldGoalsAttempted', 'defendedAtRimFieldGoalPercentage']
    team_track_stats = team_track_stats[columns_order]

    # Prepare the INSERT SQL statement
    insert_stmt = '''
    INSERT INTO TeamBoxScorePlayerTrackV3 (gameId, teamId, teamCity, teamName, teamTricode, teamSlug, minutes, distance, reboundChancesOffensive, reboundChancesDefensive, reboundChancesTotal, touches, secondaryAssists, freeThrowAssists, passes, assists, contestedFieldGoalsMade, contestedFieldGoalsAttempted, contestedFieldGoalPercentage, uncontestedFieldGoalsMade, uncontestedFieldGoalsAttempted, uncontestedFieldGoalsPercentage, fieldGoalPercentage, defendedAtRimFieldGoalsMade, defendedAtRimFieldGoalsAttempted, defendedAtRimFieldGoalPercentage)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    '''

    # Iterate over DataFrame rows and insert data
    for index, row in team_track_stats.iterrows():
        try:
            cursor.execute(insert_stmt, tuple(row))
            cnxn.commit()
        except Exception as e:
            print(f"An error occurred at row {index}: {e}")
            # Handle the error based on your requirements (break, continue, etc.)

    # Close the connection
    cursor.close()
    cnxn.close()
    del team_track_stats, track_stats, df, team


# BoxScoreScoringV3

In [97]:
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';')
cursor = cnxn.cursor()
sql = "SELECT distinct cast(FORMAT(GAME_ID, '0000000000') as char) as GAME_ID FROM [nba_game_data].[dbo].[leagueGameLog] --where yearSeason = {}"#.format(yearSeason)
all_games = pd.read_sql(sql,cnxn)
# all_games

In [98]:
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';')
cursor = cnxn.cursor()
sql = "SELECT  DISTINCT CAST(FORMAT(CAST([gameId] AS BIGINT), '0000000000') AS CHAR) AS GAME_ID FROM [nba_game_data].[dbo].[BoxScoreScoringV3]"
data = pd.read_sql(sql,cnxn)
cnxn.close()

In [99]:
score = all_games.merge(data, on=['GAME_ID'],how='left', indicator=True)
score = score.loc[score['_merge']=='left_only'].reset_index(drop=True)
score['GAME_ID'] = score['GAME_ID'].astype(str).str.zfill(10)
score

,GAME_ID,_merge
0,0022301193,left_only
1,0022301194,left_only
2,0022301190,left_only
3,0022301196,left_only
4,0022301200,left_only
5,0022301195,left_only
6,0022301187,left_only
7,0022301188,left_only
8,0022301197,left_only
9,0022301198,left_only


In [100]:
score_stats = pd.DataFrame()
team_score_stats = pd.DataFrame()
for index, row in score.iterrows():
    if index%500 == 0 and index!=0:
        print(index)
    if index%5000 == 0 and index!=0:
        print(index)
    try:
        flattened_data = []

        df = BoxScoreScoringV3(game_id = row['GAME_ID'].replace(" ",""),timeout=100).get_data_frames()
        team = df[1]
        df = df[0]
        # Create a DataFrame from the flattened data
        df['GAME_ID'] = row['GAME_ID']
        score_stats = pd.concat([score_stats,df]).reset_index(drop=True)
        team_score_stats = pd.concat([team_score_stats,team]).reset_index(drop=True)
        
    except:
        pass

In [101]:
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';')
cursor = cnxn.cursor()

# Rearrange DataFrame columns to match the BoxScoreScoringV3 table structure
columns_order = ['gameId', 'teamId', 'teamCity', 'teamName', 'teamTricode', 'teamSlug', 'personId', 'firstName', 'familyName', 'nameI', 'playerSlug', 'position', 'comment', 'jerseyNum', 'minutes', 'percentageFieldGoalsAttempted2pt', 'percentageFieldGoalsAttempted3pt', 'percentagePoints2pt', 'percentagePointsMidrange2pt', 'percentagePoints3pt', 'percentagePointsFastBreak', 'percentagePointsFreeThrow', 'percentagePointsOffTurnovers', 'percentagePointsPaint', 'percentageAssisted2pt', 'percentageUnassisted2pt', 'percentageAssisted3pt', 'percentageUnassisted3pt', 'percentageAssistedFGM', 'percentageUnassistedFGM']
scoring_stats = score_stats[columns_order]  # Replace 'advanced_stats' with your DataFrame name

# Prepare the INSERT SQL statement for BoxScoreScoringV3
insert_stmt = '''
INSERT INTO BoxScoreScoringV3 (gameId, teamId, teamCity, teamName, teamTricode, teamSlug, personId, firstName, familyName, nameI, playerSlug, position, comment, jerseyNum, minutes, percentageFieldGoalsAttempted2pt, percentageFieldGoalsAttempted3pt, percentagePoints2pt, percentagePointsMidrange2pt, percentagePoints3pt, percentagePointsFastBreak, percentagePointsFreeThrow, percentagePointsOffTurnovers, percentagePointsPaint, percentageAssisted2pt, percentageUnassisted2pt, percentageAssisted3pt, percentageUnassisted3pt, percentageAssistedFGM, percentageUnassistedFGM)
VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
'''

# Iterate over DataFrame rows and insert data
for index, row in scoring_stats.iterrows():
    try:
        cursor.execute(insert_stmt, tuple(row))
        cnxn.commit()
    except Exception as e:
        print(f"An error occurred at row {index}: {e}")

# Close the connection
cursor.close()
cnxn.close()


In [102]:
if len(team_score_stats) > 0:
    # Database connection
    cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER=' + server + ';DATABASE=' + database + ';')
    cursor = cnxn.cursor()

    # Rearrange DataFrame columns to match the SQL table structure
    columns_order = ['gameId', 'teamId', 'teamCity', 'teamName', 'teamTricode', 'teamSlug', 'minutes', 'percentageFieldGoalsAttempted2pt', 'percentageFieldGoalsAttempted3pt', 'percentagePoints2pt', 'percentagePointsMidrange2pt', 'percentagePoints3pt', 'percentagePointsFastBreak', 'percentagePointsFreeThrow', 'percentagePointsOffTurnovers', 'percentagePointsPaint', 'percentageAssisted2pt', 'percentageUnassisted2pt', 'percentageAssisted3pt', 'percentageUnassisted3pt', 'percentageAssistedFGM', 'percentageUnassistedFGM']
    team_score_stats = team_score_stats[columns_order]

    # Prepare the INSERT SQL statement
    insert_stmt = '''
    INSERT INTO TeamBoxScoreScoringV3 (gameId, teamId, teamCity, teamName, teamTricode, teamSlug, minutes, percentageFieldGoalsAttempted2pt, percentageFieldGoalsAttempted3pt, percentagePoints2pt, percentagePointsMidrange2pt, percentagePoints3pt, percentagePointsFastBreak, percentagePointsFreeThrow, percentagePointsOffTurnovers, percentagePointsPaint, percentageAssisted2pt, percentageUnassisted2pt, percentageAssisted3pt, percentageUnassisted3pt, percentageAssistedFGM, percentageUnassistedFGM)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    '''

    # Iterate over DataFrame rows and insert data
    for index, row in team_score_stats.iterrows():
        try:
            cursor.execute(insert_stmt, tuple(row))
            cnxn.commit()
        except Exception as e:
            print(f"An error occurred at row {index}: {e}")
            # Handle the error based on your requirements (break, continue, etc.)

    # Close the connection
    cursor.close()
    cnxn.close()
    del team_score_stats, scoring_stats, df, team


# PlayerBaseJoin

In [103]:
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';')
cursor = cnxn.cursor()
sql = f'''
SELECT 
        pgl.[SEASON_YEAR]
        ,pgl.[PLAYER_ID]
        ,pgl.[PLAYER_NAME]
        ,pgl.[NICKNAME]
        ,COALESCE(CASE WHEN ctr.[POSITION] = 'G-F' THEN 'G' ELSE
        CASE WHEN ctr.[POSITION] = 'F-C' THEN 'F' ELSE
        CASE WHEN ctr.[POSITION] = 'F-G' THEN 'F' ELSE
        CASE WHEN ctr.[POSITION] = 'C-F' THEN 'C' ELSE
        ctr.[POSITION] END END END END ,ctr2.POSITION,'NF') as POSITION
        ,pgl.[TEAM_ID]
        ,pgl.[TEAM_ABBREVIATION]
        ,pgl.[TEAM_NAME]
        ,pgl.[GAME_ID]
        ,pgl.[GAME_DATE]
        ,LAG (pgl.[GAME_DATE]) OVER (PARTITION BY pgl.[PLAYER_ID] ORDER BY pgl.[GAME_DATE]) AS Last_Game_Played
        ,pgl.[MATCHUP]
        ,pgl.[WL]
        ,CASE WHEN PGL.WL = 'W' THEN 1 ELSE 0 END AS WLInt
        ,LAG (CASE WHEN PGL.WL = 'W' THEN 1 ELSE 0 END) OVER (PARTITION BY pgl.[PLAYER_ID] ORDER BY pgl.[GAME_DATE]) AS did_they_win_last_game
        ,dir.total_non_starters_out
        ,dir.total_starters_out
        ,tr.after_trade
        ,tr.traded_in_last_10_games


        ,pgl.[MIN]
        ,pgl.[FGM]
        ,pgl.[FGA]
        ,pgl.[FG_PCT]
        ,pgl.[FG3M]
        ,pgl.[FG3A]
        ,pgl.[FG3_PCT]
        ,pgl.[FTM]
        ,pgl.[FTA]
        ,pgl.[FT_PCT]
        ,pgl.[OREB]
        ,pgl.[DREB]
        ,pgl.[REB]
        ,pgl.[AST]
        ,pgl.[TOV]
        ,pgl.[STL]
        ,pgl.[BLK]
        ,pgl.[BLKA]
        ,pgl.[PF]
        ,pgl.[PFD]
        ,pgl.[PTS]
        ,pgl.[PLUS_MINUS]
        ,pgl.[NBA_FANTASY_PTS]
        ,pgl.[DD2]
        ,pgl.[TD3]
        ,pgl.[WNBA_FANTASY_PTS]
        ,pgl.[GP_RANK]
        ,pgl.[W_RANK]
        ,pgl.[L_RANK]
        ,pgl.[W_PCT_RANK]
        ,pgl.[MIN_RANK]
        ,pgl.[FGM_RANK]
        ,pgl.[FGA_RANK]
        ,pgl.[FG_PCT_RANK]
        ,pgl.[FG3M_RANK]
        ,pgl.[FG3A_RANK]
        ,pgl.[FG3_PCT_RANK]
        ,pgl.[FTM_RANK]
        ,pgl.[FTA_RANK]
        ,pgl.[FT_PCT_RANK]
        ,pgl.[OREB_RANK]
        ,pgl.[DREB_RANK]
        ,pgl.[REB_RANK]
        ,pgl.[AST_RANK]
        ,pgl.[TOV_RANK]
        ,pgl.[STL_RANK]
        ,pgl.[BLK_RANK]
        ,pgl.[BLKA_RANK]
        ,pgl.[PF_RANK]
        ,pgl.[PFD_RANK]
        ,pgl.[PTS_RANK]
        ,pgl.[PLUS_MINUS_RANK]
        ,pgl.[NBA_FANTASY_PTS_RANK]
        ,pgl.[DD2_RANK]
        ,pgl.[TD3_RANK]
        ,pgl.[WNBA_FANTASY_PTS_RANK]
        ,pgl.[AVAILABLE_FLAG]
        ,pgl.[yearSeason]
        ,pgl.[oppAbrv]

        ,bsa.[estimatedOffensiveRating]
        ,bsa.[offensiveRating]
        ,bsa.[estimatedDefensiveRating]
        ,bsa.[defensiveRating]
        ,bsa.[estimatedNetRating]
        ,bsa.[netRating]
        ,bsa.[assistPercentage]
        ,bsa.[assistToTurnover]
        ,bsa.[assistRatio]
        ,bsa.[offensiveReboundPercentage]
        ,bsa.[defensiveReboundPercentage]
        ,bsa.[reboundPercentage]
        ,bsa.[turnoverRatio]
        ,bsa.[effectiveFieldGoalPercentage]
        ,bsa.[trueShootingPercentage]
        ,bsa.[usagePercentage]
        ,bsa.[estimatedUsagePercentage]
        ,bsa.[estimatedPace]
        ,bsa.[pace]
        ,bsa.[pacePer40]
        ,bsa.[possessions]
        ,bsa.[PIE]

        ,bsd.[matchupMinutes]
        ,bsd.[partialPossessions]
        ,bsd.[switchesOn]
        ,bsd.[playerPoints]
        ,bsd.[defensiveRebounds]
        ,bsd.[matchupAssists]
        ,bsd.[matchupTurnovers]
        ,bsd.[steals]
        ,bsd.[blocks]
        ,bsd.[matchupFieldGoalsMade]
        ,bsd.[matchupFieldGoalsAttempted]
        ,bsd.[matchupFieldGoalPercentage]
        ,bsd.[matchupThreePointersMade]
        ,bsd.[matchupThreePointersAttempted]
        ,bsd.[matchupThreePointerPercentage]

        ,bsh.[points]
        ,bsh.[contestedShots]
        ,bsh.[contestedShots2pt]
        ,bsh.[contestedShots3pt]
        ,bsh.[deflections]
        ,bsh.[chargesDrawn]
        ,bsh.[screenAssists]
        ,bsh.[screenAssistPoints]
        ,bsh.[looseBallsRecoveredOffensive]
        ,bsh.[looseBallsRecoveredDefensive]
        ,bsh.[looseBallsRecoveredTotal]
        ,bsh.[offensiveBoxOuts]
        ,bsh.[defensiveBoxOuts]
        ,bsh.[boxOutPlayerTeamRebounds]
        ,bsh.[boxOutPlayerRebounds]
        ,bsh.[boxOuts]

        ,bsm.[pointsOffTurnovers]
        ,bsm.[pointsSecondChance]
        ,bsm.[pointsFastBreak]
        ,bsm.[pointsPaint]
        ,bsm.[oppPointsOffTurnovers]
        ,bsm.[oppPointsSecondChance]
        ,bsm.[oppPointsFastBreak]
        ,bsm.[oppPointsPaint]
        ,bsm.[blocksAgainst]
        ,bsm.[foulsPersonal]
        ,bsm.[foulsDrawn]

        ,bss.[percentageFieldGoalsAttempted2pt]
        ,bss.[percentageFieldGoalsAttempted3pt]
        ,bss.[percentagePoints2pt]
        ,bss.[percentagePointsMidrange2pt]
        ,bss.[percentagePoints3pt]
        ,bss.[percentagePointsFastBreak]
        ,bss.[percentagePointsFreeThrow]
        ,bss.[percentagePointsOffTurnovers]
        ,bss.[percentagePointsPaint]
        ,bss.[percentageAssisted2pt]
        ,bss.[percentageUnassisted2pt]
        ,bss.[percentageAssisted3pt]
        ,bss.[percentageUnassisted3pt]
        ,bss.[percentageAssistedFGM]
        ,bss.[percentageUnassistedFGM]

        ,bst.[speed]
        ,bst.[distance]
        ,bst.[reboundChancesOffensive]
        ,bst.[reboundChancesDefensive]
        ,bst.[reboundChancesTotal]
        ,bst.[touches]
        ,bst.[secondaryAssists]
        ,bst.[freeThrowAssists]
        ,bst.[passes]
        ,bst.[assists]
        ,bst.[contestedFieldGoalsMade]
        ,bst.[contestedFieldGoalsAttempted]
        ,bst.[contestedFieldGoalPercentage]
        ,bst.[uncontestedFieldGoalsMade]
        ,bst.[uncontestedFieldGoalsAttempted]
        ,bst.[uncontestedFieldGoalsPercentage]
        ,bst.[fieldGoalPercentage]
        ,bst.[defendedAtRimFieldGoalsMade]
        ,bst.[defendedAtRimFieldGoalsAttempted]
        ,bst.[defendedAtRimFieldGoalPercentage]


  FROM [nba_game_data].[dbo].[PlayerGameLogs] pgl

  LEFT JOIN nba_game_data.dbo.BoxScoreAdvancedV3 bsa
  on
  pgl.GAME_ID = bsa.GAME_ID
  and pgl.PLAYER_ID = bsa.personId

  LEFT JOIN nba_game_data.dbo.[BoxScoreDefensiveV2] bsd
  on
  pgl.GAME_ID = cast(bsd.gameid as int)
  and pgl.PLAYER_ID = bsd.personId

  LEFT JOIN nba_game_data.dbo.[BoxScoreHustleV2] bsh
  on
  pgl.GAME_ID = cast(bsh.gameid as int)
  and pgl.PLAYER_ID = bsh.personId
  
  LEFT JOIN nba_game_data.dbo.[BoxScoreMiscV3] bsm
  on
  pgl.GAME_ID = cast(bsm.gameid as int)
  and pgl.PLAYER_ID = bsm.personId

  LEFT JOIN nba_game_data.dbo.[BoxScoreScoringV3] bss
  on
  pgl.GAME_ID = cast(bss.gameid as int)
  and pgl.PLAYER_ID = bss.personId

  LEFT JOIN nba_game_data.dbo.[BoxScorePlayerTrackV3] bst
  on
  pgl.GAME_ID = cast(bst.gameid as int)
  and pgl.PLAYER_ID = bst.personId

   LEFT OUTER JOIN [nba_game_data].[dbo].[CommonTeamRoster] ctr
  ON
  cast(pgl.PLAYER_ID as int) = cast(ctr.PLAYER_ID as int)
  AND cast(pgl.TEAM_ID as int) = cast(ctr.TeamID as int)
  AND cast(pgl.yearSeason as int) = cast(ctr.SEASON as int)
  
  LEFT OUTER JOIN [nba_game_data].[dbo].[BackupPlayerPosition] ctr2
  ON 
  cast(pgl.PLAYER_ID as int) = cast(ctr2.PLAYER_ID as int)
  
  left outer join [nba_game_data].[dbo].[dailyInjuryReportV1] dir
  on 
  pgl.TEAM_ID = dir.TEAM_ID 
  and pgl.GAME_ID = dir.GAME_ID
  
  left outer join [nba_game_data].[dbo].[TradeReportV1] tr
  on 
  pgl.GAME_ID = tr.GAME_ID
  and pgl.PLAYER_ID = tr.PLAYER_ID
  
  WHERE pgl.yearSeason = '{yearSeason}' 
  
'''
player_games = pd.read_sql(sql,cnxn)

In [104]:
player_games = player_games.replace(np.nan, None)

In [105]:
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';')
cursor = cnxn.cursor()

# Example DELETE statement, adjust as needed
cursor.execute(f"DELETE FROM BasePlayer WHERE yearSeason = '{yearSeason}' ")
cnxn.commit()

In [106]:
# Rearrange DataFrame columns to match the BasePlayer table structure
columns_order = [
    'SEASON_YEAR', 'PLAYER_ID', 'PLAYER_NAME', 'NICKNAME', 'POSITION', 'TEAM_ID', 'TEAM_ABBREVIATION','TEAM_NAME','GAME_ID', 'GAME_DATE', 'Last_Game_Played','MATCHUP', 'WL', 'WLInt', 'did_they_win_last_game','total_non_starters_out','total_starters_out','MIN', 'FGM', 'FGA', 'FG_PCT','after_trade','traded_in_last_10_games' , 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB','AST', 'TOV', 'STL', 'BLK', 'BLKA', 'PF', 'PFD', 'PTS', 'PLUS_MINUS', 'NBA_FANTASY_PTS', 'DD2', 'TD3','WNBA_FANTASY_PTS', 'GP_RANK', 'W_RANK', 'L_RANK', 'W_PCT_RANK', 'MIN_RANK', 'FGM_RANK', 'FGA_RANK','FG_PCT_RANK', 'FG3M_RANK', 'FG3A_RANK', 'FG3_PCT_RANK', 'FTM_RANK', 'FTA_RANK', 'FT_PCT_RANK', 'OREB_RANK','DREB_RANK', 'REB_RANK', 'AST_RANK', 'TOV_RANK', 'STL_RANK', 'BLK_RANK', 'BLKA_RANK', 'PF_RANK', 'PFD_RANK','PTS_RANK', 'PLUS_MINUS_RANK', 'NBA_FANTASY_PTS_RANK', 'DD2_RANK', 'TD3_RANK', 'WNBA_FANTASY_PTS_RANK','AVAILABLE_FLAG', 'yearSeason', 'oppAbrv', 'estimatedOffensiveRating', 'offensiveRating','estimatedDefensiveRating', 'defensiveRating', 'estimatedNetRating', 'netRating', 'assistPercentage','assistToTurnover', 'assistRatio', 'offensiveReboundPercentage', 'defensiveReboundPercentage', 'reboundPercentage','turnoverRatio', 'effectiveFieldGoalPercentage', 'trueShootingPercentage', 'usagePercentage','estimatedUsagePercentage', 'estimatedPace', 'pace', 'pacePer40', 'possessions', 'PIE', 'matchupMinutes','partialPossessions', 'switchesOn', 'playerPoints', 'defensiveRebounds', 'matchupAssists', 'matchupTurnovers','steals', 'blocks', 'matchupFieldGoalsMade', 'matchupFieldGoalsAttempted', 'matchupFieldGoalPercentage','matchupThreePointersMade', 'matchupThreePointersAttempted', 'matchupThreePointerPercentage', 'points','contestedShots', 'contestedShots2pt', 'contestedShots3pt', 'deflections', 'chargesDrawn', 'screenAssists','screenAssistPoints', 'looseBallsRecoveredOffensive', 'looseBallsRecoveredDefensive', 'looseBallsRecoveredTotal','offensiveBoxOuts', 'defensiveBoxOuts', 'boxOutPlayerTeamRebounds', 'boxOutPlayerRebounds', 'boxOuts','pointsOffTurnovers', 'pointsSecondChance', 'pointsFastBreak', 'pointsPaint', 'oppPointsOffTurnovers','oppPointsSecondChance', 'oppPointsFastBreak', 'oppPointsPaint', 'blocksAgainst', 'foulsPersonal', 'foulsDrawn','percentageFieldGoalsAttempted2pt', 'percentageFieldGoalsAttempted3pt', 'percentagePoints2pt','percentagePointsMidrange2pt', 'percentagePoints3pt', 'percentagePointsFastBreak', 'percentagePointsFreeThrow','percentagePointsOffTurnovers', 'percentagePointsPaint', 'percentageAssisted2pt', 'percentageUnassisted2pt','percentageAssisted3pt', 'percentageUnassisted3pt', 'percentageAssistedFGM', 'percentageUnassistedFGM', 'speed','distance', 'reboundChancesOffensive', 'reboundChancesDefensive', 'reboundChancesTotal', 'touches', 'secondaryAssists','freeThrowAssists', 'passes', 'assists', 'contestedFieldGoalsMade', 'contestedFieldGoalsAttempted','contestedFieldGoalPercentage', 'uncontestedFieldGoalsMade', 'uncontestedFieldGoalsAttempted', 'uncontestedFieldGoalsPercentage','fieldGoalPercentage', 'defendedAtRimFieldGoalsMade', 'defendedAtRimFieldGoalsAttempted', 'defendedAtRimFieldGoalPercentage'
]
player_games = player_games[columns_order]  # Replace 'your_dataframe' with the name of your DataFrame

# player_games['total_non_starters_out'] = player_games['total_non_starters_out'].astype(int)
# player_games['total_starters_out'] = player_games['total_starters_out'].astype(int)

# Prepare the INSERT SQL statement for BasePlayer
insert_stmt = '''
INSERT INTO BasePlayer ([SEASON_YEAR],[PLAYER_ID],[PLAYER_NAME],[NICKNAME],[POSITION],[TEAM_ID],[TEAM_ABBREVIATION],[TEAM_NAME],[GAME_ID],[GAME_DATE],[Last_Game_Played],[MATCHUP],[WL],[WLInt],[did_they_win_last_game],[total_non_starters_out],[total_starters_out],[after_trade],[traded_in_last_10_games],[MIN],[FGM],[FGA],[FG_PCT],[FG3M],[FG3A],[FG3_PCT],[FTM],[FTA],[FT_PCT],[OREB],[DREB],[REB],[AST],[TOV],[STL],[BLK],[BLKA],[PF],[PFD],[PTS],[PLUS_MINUS],[NBA_FANTASY_PTS],[DD2],[TD3],[WNBA_FANTASY_PTS],[GP_RANK],[W_RANK],[L_RANK],[W_PCT_RANK],[MIN_RANK],[FGM_RANK],[FGA_RANK],[FG_PCT_RANK],[FG3M_RANK],[FG3A_RANK],[FG3_PCT_RANK],[FTM_RANK],[FTA_RANK],[FT_PCT_RANK],[OREB_RANK],[DREB_RANK],[REB_RANK],[AST_RANK],[TOV_RANK],[STL_RANK],[BLK_RANK],[BLKA_RANK],[PF_RANK],[PFD_RANK],[PTS_RANK],[PLUS_MINUS_RANK],[NBA_FANTASY_PTS_RANK],[DD2_RANK],[TD3_RANK],[WNBA_FANTASY_PTS_RANK],[AVAILABLE_FLAG],[yearSeason],[oppAbrv],[estimatedOffensiveRating],[offensiveRating],[estimatedDefensiveRating],[defensiveRating],[estimatedNetRating],[netRating],[assistPercentage],[assistToTurnover],[assistRatio],[offensiveReboundPercentage],[defensiveReboundPercentage],[reboundPercentage],[turnoverRatio],[effectiveFieldGoalPercentage],[trueShootingPercentage],[usagePercentage],[estimatedUsagePercentage],[estimatedPace],[pace],[pacePer40],[possessions],[PIE],[matchupMinutes],[partialPossessions],[switchesOn],[playerPoints],[defensiveRebounds],[matchupAssists],[matchupTurnovers],[steals],[blocks],[matchupFieldGoalsMade],[matchupFieldGoalsAttempted],[matchupFieldGoalPercentage],[matchupThreePointersMade],[matchupThreePointersAttempted],[matchupThreePointerPercentage],[points],[contestedShots],[contestedShots2pt],[contestedShots3pt],[deflections],[chargesDrawn],[screenAssists],[screenAssistPoints],[looseBallsRecoveredOffensive],[looseBallsRecoveredDefensive],[looseBallsRecoveredTotal],[offensiveBoxOuts],[defensiveBoxOuts],[boxOutPlayerTeamRebounds],[boxOutPlayerRebounds],[boxOuts],[pointsOffTurnovers],[pointsSecondChance],[pointsFastBreak],[pointsPaint],[oppPointsOffTurnovers],[oppPointsSecondChance],[oppPointsFastBreak],[oppPointsPaint],[blocksAgainst],[foulsPersonal],[foulsDrawn],[percentageFieldGoalsAttempted2pt],[percentageFieldGoalsAttempted3pt],[percentagePoints2pt],[percentagePointsMidrange2pt],[percentagePoints3pt],[percentagePointsFastBreak],[percentagePointsFreeThrow],[percentagePointsOffTurnovers],[percentagePointsPaint],[percentageAssisted2pt],[percentageUnassisted2pt],[percentageAssisted3pt],[percentageUnassisted3pt],[percentageAssistedFGM],[percentageUnassistedFGM],[speed],[distance],[reboundChancesOffensive],[reboundChancesDefensive],[reboundChancesTotal],[touches],[secondaryAssists],[freeThrowAssists],[passes],[assists],[contestedFieldGoalsMade],[contestedFieldGoalsAttempted],[contestedFieldGoalPercentage],[uncontestedFieldGoalsMade],[uncontestedFieldGoalsAttempted],[uncontestedFieldGoalsPercentage],[fieldGoalPercentage],[defendedAtRimFieldGoalsMade],[defendedAtRimFieldGoalsAttempted],[defendedAtRimFieldGoalPercentage])
VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)  
'''

# Iterate over DataFrame rows and insert data
for index, row in player_games.iterrows():
    try:
        cursor.execute(insert_stmt, tuple(row))
        cnxn.commit()
        
    except Exception as e:
        print(f"An error occurred at row {index}: {e}")
        

# Close the connection
cursor.close()
cnxn.close()


In [107]:
# time.sleep(30)
# # This command will put the computer to sleep in Windows
# os.system("rundll32.exe powrprof.dll,SetSuspendState 0,1,0")
